# PRÁCTICA 2 - ALGORITMO HEURÍSTICOS NO CONSTRUCTIVOS

# Funciones Base

## Carga de datos

In [133]:
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

In [134]:
def movements_to_list(deltas_df):
    """
    Devuelve array con todos los movimientos realizados en las estaciones. Cada posicion del array esta compuesto por el id de la estacion y la cantidad de bicis retiradas/aniadidas en cada movimiento
    
    Parametros
        deltas_df : DataFrame
            Pandas DataFrame a convertir
    Return
        Numpy array
    """
    move_list = np.array([]) # Inicializamos el array
    rows = len(deltas_df.index) # Mediciones totales realizadas
    columns = len(deltas_df.columns) # Numero de estaciones
    
    # Recorreremos todas las mediciones e iremos transformando todas ellas que sean distintas de 0
    for i in range(0, rows):
        if i != 0:
            row = deltas_df.iloc[i] # Seleccionamos una medicion
            selec = row[row != 0] # Seleccionamos que estaciones han sufrido algun cambio
            for j in range(0, len(selec)):
                lst = np.array([[int(selec.index[j]), selec[j]]]) # Obtenemos un array con el index y el numero de bicis retiradas/aniadidas
                # En la setencia anterior, el primer corchete sirve para crear el array. Mientras que el segundo para crear una tupla tal que: [estacion, movements]
                if len(move_list) == 0:
                    move_list = lst # Asignamos la primera tupla como 
                else:
                    move_list = np.append(move_list, lst, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
    
    return move_list

In [135]:
def update_dataframe(df, df_columns, seed, algorithm, km, t, solution):
    """
    Actualiza un data frame ya creado
    
    Parametros
        df : Pandas DataFrame
            DataFrame a actualizar
        df_columns : list
            Nombre de las columnas del DataFrame
        seed : int
            Semilla utilizada en el algoritmo
        algorithm : str
            Nombre del algoritmo utilizado
        km : float
            Kilometros obtenidos por el algoritmo
        t : float
            Tiempo demorado por el agoritmo en ejecutarse
        solution : Numpy ndarray
            Vector solucion obtenido por el algoritmo
    
    Return 
        Devuelve una copia del DataFrame ya actualizado
    """
    
    data = [(seed, algorithm, km, t, solution)]
    new_df = pd.DataFrame(data, columns=df_columns)
    return df.append(new_df) # Le aniadimos a al dataframe la nueva fila y lo devolvemos

## Función de evaluación

In [136]:
def evaluate(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''
    
    # PARAMETROS DE EJECUCION
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)

    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    bicycles = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * bicycles * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] -= bicycles # Le restamos el numero de bicicletas necesario
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
                        
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion/estaciones mas cercanas con capacidad suficiente de slots

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    slots = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * slots # Kms hacia la estacion * numero de slots a buscar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] += slots # Le sumamos el numero de bicicletas dejas en los slots libres
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
    
    '''
    Debemos hacer un ajuste al resultado en funcion del numero de plazas de la solucion. En caso de ser muy cercano a 205(minimo de slots), debemos penalizar poco
    en caso de ser muy superior, debemos penalizar mucho. Para ello tendremos un factor de coste alpha, que deberemos estimar nosotros. El objetivo es que el numero de slots
    se acerquen los maximo posible al minimo(205)
    '''
    # PARAMETROS PARA EL CALCULO FINAL
    
    min_slots = 205
    max_slots = 305
    epsilon = 2.0
    
    capacity_slots = sum(capacity)
    alpha = (capacity_slots-min_slots)**epsilon / max_slots

    # Calculamos los nuevos kilometros
    tkms = tkms + alpha*capacity.sum() # ALPHA*SLOTS
    
    return round(tkms,3) # Redondeamos a 3 decimales

In [137]:
def evaluate_no_alpha(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''

    # PARAMETROS DE EJECUCION
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)

    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    bicycles = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * bicycles * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] -= bicycles # Le restamos el numero de bicicletas necesario
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
                        
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion/estaciones mas cercanas con capacidad suficiente de slots

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    slots = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * slots # Kms hacia la estacion * numero de slots a buscar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] += slots # Le sumamos el numero de bicicletas dejas en los slots libres
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
    
    
    return round(tkms,3) # Redondeamos a 3 decimales

## Encontrar estación más cercana

In [138]:
def find_nearest_station_index(index_m, station, actual_state, capacity, search, search_type):
    """
    Devuelve una lista con los siguientes valores:
        * index-columna: Index de la estacion mas cercana con respecto al parametro station en la variable index_m
        * slots/bicicletas: Numero de slots/bicicletas aniadidas/quitadas en la estacion index-columna
    
    Parametros
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        station : int
            Estacion desde la que buscar su estacion vecina mas proxima
        actual_state : Numpy ndarray
            Array con el estado actual de bicicletas en cada estacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        search : int
            Numero de bicicletas/slots a buscar
        search_type : str
            Cadena de caracteres que nos indica el tipo de elemento a buscar: 'bicycles' o 'slots'
            
    Return
        Devuelve una lista con los indices de las estaciones en la matriz index_m y el numero de biciletas/slots cogidos en cada una de las estaciones. Ejemplo:
        [[station_index=3, slots=2], [station_index=12, bicicletas=4]]
    """
    # Valor que almacena la posicion (columna) de la estacion mas cercana con respecto al parametro station
    row_selec = index_m[station] # Seleccionamos el array de estaciones cercanas de la estacion station
    index_lst = np.array([]) # Lista donde guardaremos el index de la estaciones mas cercanas, junto al numero de bicicletas/slots asignados
    remaining = search # Numero de slots que aun faltan por asignar
    
    # Comprobamos que tipo de busqueda estamos realizando, 'bicycles' o 'slots'
    if search_type == 'bicycles':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con bicicletas suficientes
        for j in range(1, len(row_selec)):
            # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con las bicicletas suficientes
            free_bicycles = actual_state[row_selec[j]]
            
            # Si la estacion tiene biciclestas disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_bicycles > 0:
                # Cuando el numero de bicicletas de la estacion no suple la demanda
                if remaining > free_bicycles:
                    remaining = remaining - free_bicycles # Actualizamos el numero de bicicletas que faltan por asignar
                    taken = free_bicycles # Debemos tomar estas bicis
                # Cuando el numero de bicicletas de la estacion suple la demanda
                else:
                    taken = remaining # Del numero de bicicletas total disponible. Solo tomamos los necesarios
                    remaining = 0              
                
                index = np.array([[j, taken]]) # Creamos la variable que contiene la estacion y el numero 
                
                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
                    
                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos 
            
        raise Exception(f"Error, ninguna estacion tiene bicicletas. La solucion tiene {capacity.sum()} slots")
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            #if search <= actual_state[row_selec[j]]:
            #    return j # Devolvemos el index
            
    elif search_type == 'slots':
        # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con los slots suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            free_slots = capacity[row_selec[j]] - actual_state[row_selec[j]] # Calculamos el numero de slots disponibles restandole a la capacidad total el numero de bicicletas actuales
            
            # Si la estacion tiene slots disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_slots > 0:
                # Cuando el numero de slots de la estacion no suple la demanda o es igual que la demanda
                if remaining > free_slots:
                    remaining = remaining - free_slots # Actualizamos los slots que faltan por asignar
                    taken = free_slots # Debemos tomar estos slots

                # Cuando el numero de slots de la estacion si suple la demanda
                else:
                    taken = remaining # Del numero de slots total disponible. Solo tomamos los necesarios
                    remaining = 0
                
                index = np.array([[j, taken]])

                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list

                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos    
        
        raise Exception(f"Error, todas las estaciones estan llenas. La solucion tiene {capacity.sum()} slots")
    else:
        raise Exception("La variable search_type no contiene un tipo valido") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes

## Mostrar variación (Función no necesaria)

In [139]:
def show_variation(init_solution, init_tkms, new_solution, new_tkms):
    """
    Muestra por pantalla la diferencia de slots existentes entre dos soluciones

    Parametros
        init_solution : Numpy ndarray
            Solucion inicial a comparar
        init_tkms : float
            Numero de kilometros recorridos en la solucion inicial
        new_solution : Numpy ndarray
            Solucion a comparar con respecto a init_solution
        new_tkms : float
            Numero de kilometros recorridos en la nueva solucion
    """

    columns = np.arange(0,len(init_solution))
    variation = init_solution - new_solution

    # Para representarlo carrectamente la variacion crearemos un string con los espacios correspondientes
    string = '['
    space = ' '
    for i in range(0,len(init_solution)):
        # Es caso de que la variacion sea mayor que 0 y menor a 10, sera necesario aniadir un espacio
        if 10 > variation[i] and variation[i] >= 0:
            if variation [i] == 0: # Cuando aniadimos un 0 debemos agregar un espacio delante. Cuando es un numero positivo, un signo +
                string += space + str(variation[i])
            else:
                string += '+' + str(variation[i])
        else:
            string += str(variation[i])


        if i != len(init_solution)-1:
            string += space

    string += ']'

    print('')
    print('----------------------COMPARACION DE SOLUCIONES-----------------------')
    print('    Estacion    : {}'.format(columns))
    print('----------------------------------------------------------------------')
    print('Solucion inicial: {} - {} km'.format(init_solution, init_tkms))
    print(' Solucion final : {} - {} km'.format(new_solution, new_tkms))
    print('----------------------------------------------------------------------')
    print('    Variacion   : ' + string)
    print('----------------------------------------------------------------------')
    print('')

## Generación de la solución inicial

### GREEDY

In [140]:
def generate_initial_greedy_solution(init_state, max_slots):
    '''
    Crea una solucion inicial para el algoritmo greedy a partir del numero maximo de slots disponibles y el estado inicial
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de bicletas en las estaciones
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    stations = len(init_state)
    solution = init_state.copy() # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    proportion = max_slots / init_state.sum() # Nueva proporcion para el maximo de slots
    
    # station * proportion
    solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
    return solution
    

### EL RESTO DE ALGORITMOS

In [141]:
def generate_initial_solution(init_state, max_slots):
    '''
    Crea una solucion a partir del estado inicial y el numero maximo de slots disponibles
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    n_stations = len(init_state)
    
    # 2 3 4 3 2 4 5 2 5
    # 2 3 4 3 2 9 4 1 3
    
    # Usaremos un metodo parecido al usado en la generacion greedy. Generaremos N numeros aleatorios entre min_value y max_value, multiplicamos
    # cada uno por la proporcion entre max_slots/(la suma de todos ellos) y redondear por defecto. 
    
    max_value = 10
    min_value = 2
    margin = 20 # Margen para hacer que la solucion generada sea compatible con el estado actual
    accept = False # Varible de control
    
    while accept == False:
        solution = np.random.randint(min_value, max_value, n_stations) # Crea un array de numeros aleatorios con el tamanio del numero de estaciones
        # Haremos que la proporcion no este correlacionada exactammente a max_slots, estableceremos un valor mas pequenio para luego poder asignar valores
        # a los las estaciones que no sean compatibles con la solucion inicial
        proportion = (max_slots-margin) / solution.sum() # Nueva proporcion para el maximo de slots
        
        # station * proportion
        solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
        amount = solution.sum() 
        remaining = max_slots - amount # Slots restantes por asignar
        
        # Comprobamos a que estacion le faltan slots para satisfacer el estado inicial
        lst = solution < init_state
        for i in range(0,len(lst)):
            if lst[i] == True:
                rest = init_state[i] - solution[i]
                if (remaining - rest) >= 0: 
                    solution[i] += rest # Aniadimos el numero de slots necesarios
                    remaining -= rest # Le quitamos los aniadidos al total restante
                else:
                    break # Si no hay slots suficientes
        
        # Si faltan slots por asignar recorreremos el array asignando slots
        if remaining >= 0:
            # Ahora que tenemos una solucion aceptable aniadimos los slots restantes. Vamos aniadiendo a cada estacion los valores 1 a 1
            i = 0
            while remaining > 0:
                solution[i] += 1 
                remaining -= 1
                i += 1
                if i == n_stations:
                    i = 0
            
            check = solution > init_state
            # Si todas los estaciones de la solucion tienen los slots suficientes para albergar a la solucion inicial. Entonces se acepta
            if check.sum() == n_stations:
                accept = True 

    return solution.astype(int)

In [142]:
def adjust_solution(solution, init_state):
    """
    Ajusta una solucion al estado inicial para que el numero de slots minimo de la solucion coincida con al menos la capacidad del estado inicial
    
    Parametros
        solution : Numpy ndarray
            Vector solucion con las capacidades de las estaciones
        init_state : Numpy ndarray
            Vector con el numero de bicicletas de cada estacion en el estado inicial
            
    Return 
        Devuelve la solucion ajustada al estado inicial
    """
    new_solution = solution.copy()
    
    # Primero comprobamos que el numero de slots sea el minimo (205)
    '''
    proportion = max_slots / init_state.sum() # Nueva proporcion para el maximo de slots
    
    # station * proportion
    solution = np.around(solution * proportion).astype(int) 
    '''
    if new_solution.sum() < 205:
        proportion = 210 / new_solution.sum() # Nueva proporcion para el maximo de slots
        new_solution = np.around(new_solution * proportion).astype(int) 
    
    # Ahora miramos si hay alguna estacion que tiene menos slots que bicis en el estado inicial
    
    #n_stations = len(init_state)
    slots = 4 # Numero de slots a mover
    check_iterations = 15 # Reducimos el numero de slots a mover cada 15 iteraciones
    i = 0 
    check = init_state <= new_solution # Comprobamos que estaciones de la solucion no tienen los slots suficientes
    
    while check.min() == False: # Con check.min() comprobamos si hay alguna estacion que tenga mas bicis que slots
        '''
        Se puede dar el caso en el que moviendo 4 slots nunca se pueda ajustar el vector solution al estado inicial. Por eso, cada cierto numero de 
        iteraciones lo reducimos
        '''
        if ((i % check_iterations) == 0) and (i != 0) and (slots > 1):
            slots -= 1 # Reducimos el numero de slots a mover
        
        # Seleccionamos la estacion que no tenga suficientes slots
        mi = check.argmin() # Estacion con menos vecinos
        
        # Seleccionamos la estacion que tenga mas slots de sobra
        difference = new_solution - init_state
        ma = difference.argmax() # Estacion con mas vecinos
        # Comprobamos que la estacion que necesita vecinos y la que tiene mas vecinos sean distintas
        if ma == mi:
            # Debemos de buscar la segunda estacion con mas slots
            copy = new_solution.copy() # Crearemos una copia para poder encontrar el segundo mejor valor
            copy[ma] = 0 # Establecemos que el valor mas grande ahora tiene 0 slots
            difference = copy - init_state
            ma = difference.argmax() # Estacion con mas vecinos

        
        search = init_state[mi] - new_solution[mi]
        
        # Para no quitar demasiado de una estacion, establecemos un numero maximo de slots a quitar
        if search > slots:
            search = slots
        
        # Le quitamos 1 a la que mas y se lo asignamos a la que menos
        new_solution[ma] -= search
        new_solution[mi] += search
        
        check = init_state <= new_solution # Comprobamos que estaciones de la solucion no tienen los slots suficientes
        i += 1 # Aumentamos el numero de iteraciones
        
    return new_solution
        

## Movement_operator

### Movement_operator común

In [143]:
def movement_operator(actual_state, actual_solution, n_slots, neighbours_map, init_index, index):
    """
    Operador de movimiento que nos devuelve un vecino. Usando el mapa de vecinos y la variable init_index nos devuelve el primer vecino valido. Ademas, junto al vecino
    nos devuelve el siguiente vecino a buscar en caso de que el vecino encontrado previamente no satisfada nuestros requisitos. 

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
        init_index : int
            Posicion inicial por donde se debe empezar a buscar vecinos en el mapa de vecinos
        index : int
            Siguiente vecino a comprobar
    Return
        Devuelve una lista de dos posiciones. En la primera posicion se guarda el vecino encontrado y en la segunda posicion el siguiente vecino a buscar. En caso de que se haya explorado
        todos los vecinos posibles se devolvera la siguiente lista: [None, -1]. Indicando que el espacio de busqueda ya ha sido explorado
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    actual_index = index # Seleccionamos el primer vecino a explorar
    neighbour_finded = False # Contador de total de vecinos visitados

    neighbours_n = 0 # Numero de vecinos encontrados
    
    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (index - 1), si se exploran todos los posibles salimos del bucle
    while neighbour_finded == False and (init_index-1) != actual_index:
        i = neighbours_map[actual_index][0] # Estacion i
        j = neighbours_map[actual_index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbour_finded = True # Hemos encontrado un vecino

            neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        actual_index += 1
        if actual_index == last_neighbours:
            actual_index = 0 # Hacemos que siga explorando desde el principio del mapa

    if (init_index-1) == actual_index:
        neighbour = actual_solution.copy()
        actual_index = -1

    # Devolvemos la lista de vecinos encontrados
    return [neighbour, actual_index]

In [144]:
def create_neigbours_map(n_stations):
    """
    Crea una lista con todas las combinaciones posibles de estaciones. En dicha lista, se guardan los indices de cada estacion
    
    Parametros
        n_stations : int
            Numero de estaciones
    
    Return 
        Devuelve un Numpy ndarray con la siguiente estructura. Lista: [[estacion_i=1, estacion_j=2],[1,3],[1,4], ...]
    """
    neighbours_map = np.array([])
    for i in range(1, n_stations):
        for j in range(i, n_stations):
            if len(neighbours_map) != 0:
                neighbours_map = np.append(neighbours_map,np.array([[i,j]]), axis=0)
            else:
                neighbours_map = np.array([[i,j]])
                
    return neighbours_map

### Movement operator s

In [145]:
def movement_sub_list(actual_state, actual_solution, n_slots, s):
    """
    Operador de movimiento usado en el algoritmo Busqueda por entorno (VNS).
    Operador de movimiento que nos devuelve un vecino. Este vecino se generara seleccionando tantas estaciones como 
    se hayan especificado, y movimiendo slots entre las estaciones seleccionadas. 

    A partir de las estaciones seleccionadas, formaremos parejas de las estaciones consecutivas unas con otras. Es decir, 
    si se han seleccionado 8 (s=8) estaciones
    Ejemplo estaciones seleccionadas: (4,6,7,2,12,10,3,1)
    Se formaran 4 parejas:
    Ejemplo parejas resultantes: (4,6) (7,2) (12,10) (3,1)

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        s : int
            Numero de estaciones a seleccionar para crear un nuevo vecino
        
    Return 
        Vector solucion vecino
    """
    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles
    stations_selected = randomlist = np.random.randint(0,n_stations,s) # Generamos una lista de estaciones. Siempre es multiplo de 4

    neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
    a = 0 # Nos permitira seleccionar una estacion y su pareja
    while a < s:
        # Seleccionamos una pareja de estaciones
        i = stations_selected[a]
        j = stations_selected[a + 1]
        
        # Comprobamos si dicho cambio es posible
        if neighbour[i] >= n_slots and (neighbour[i]-n_slots) >= actual_state[i]:
            neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'
        
        a += 2 # Pasamos a la siguiente pareja
        
    return neighbour
    

# Algoritmos

## Búsqueda Local

In [146]:
def local_search(seed, max_slots, n_slots, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion usando el algoritmo de busqueda local. Se implementa el esquema de primero mejor vecino
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    
    # Historial de resultados
    kms_history = np.array([])
    slots_history = np.array([])
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    if display == True:
        print('LS init solution: {} - {} kms - {} slots'.format(init_solution, init_tkms, sum(init_solution)))
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    # Inicializamos las variables
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_neighbour = None
    best_tkms = None
    
    neighbour_tkms = None
    
    # Variables de control
    max_i = 3000 # Numero maximo de intentos a realizar
    i = 0 # Iteracion actual
    best_neighbour_finded = True
    
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    init_index = 0 # Primer vecino a explorar en un vecindario
    actual_index = 0 # Vecino actual a buscar en un vecindario
    
    # Buscamos mientras no superemos el numero maximo de busquedas (max_calls) y siempre que encontremos una solucion mejor a la actual en la nueva iteracion
    while i < max_i and best_neighbour_finded == True:
        # Guardamos el mejor vecino encontrado
        best_neighbour = actual_solution
        best_neighbour_tkms = actual_tkms
        
        # Generamos los vecinos de la solucion actual
        rand = np.random.randint(1,max_possible_neighbours) # Establecemos un vecino por el cual empezar a explorar
        init_index = rand # Vecino inicial explorado
        actual_index = rand # Vecino a explorar
        while True:
            neighbour, actual_index = movement_operator(init_state, actual_solution, n_slots, neighbours_map, init_index, actual_index) # Movement_operator devuelve una lista
            
            # Debemos comprobar que no hemos llegado al final de la lista
            if actual_index != -1: # Si hemos encontrado un nuevo vecino
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones

                # Si el vecino encontrado es mejor que nuestra mejor solucion
                if best_neighbour_tkms > neighbour_tkms:
                    best_neighbour = neighbour.copy()
                    best_neighbour_tkms = neighbour_tkms
                    break # Estamos buscando el primer mejor vecino, por lo tanto, salimos del bucle
            else: # Si hemos explorado todos los vecinos dejamos de buscar
                break

        
        # Comprobamos si la solucion obtenida por el vecino es mejor que nuestra solucion actual
        if best_neighbour_tkms < actual_tkms: 
            # Guardamos al vecino como la nueva solucion actual y seguimos buscando
            actual_solution = best_neighbour.copy()
            actual_tkms = best_neighbour_tkms
            if display == True:
                print('New LS best solution: {} - {} kms - {} slots'.format(actual_solution, actual_tkms, sum(actual_solution)))
        else:
            best_neighbour_finded = False
            
        i += 1 # Nueva iteracion
        
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
       
    return [actual_solution, actual_tkms, ev]           

In [147]:
def local_search_vns(seed, max_slots, n_slots, init_state, init_solution, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion usando el algoritmo de busqueda local. Se implementa el esquema de primero mejor vecino
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        init_solution : Numpy ndarray
            Array con el numero de slots que tiene cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    #init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    if display == True:
        print('LS init solution: {} - {} kms - {} slots'.format(init_solution, init_tkms, sum(init_solution)))
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos. El numero maximo de vecinos a crear para 16 estaciones es 120
    
    # Inicializamos las variables
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_neighbour = None
    best_tkms = None
    
    neighbour_tkms = None
    
    # Variables de control
    max_i = 3000 # Numero maximo de intentos a realizar
    i = 0 # Iteracion actual
    best_neighbour_finded = True
    
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    init_index = 0 # Primer vecino a explorar en un vecindario
    actual_index = 0 # Vecino actual a buscar en un vecindario
    
    # Buscamos mientras no superemos el numero maximo de busquedas (max_calls) y siempre que encontremos una solucion mejor a la actual en la nueva iteracion
    while i < max_i and best_neighbour_finded == True:
        # Guardamos el mejor vecino encontrado
        best_neighbour = actual_solution
        best_neighbour_tkms = actual_tkms
        
        # Generamos los vecinos de la solucion actual
        rand = np.random.randint(1,max_possible_neighbours) # Establecemos un vecino por el cual empezar a explorar
        init_index = rand # Vecino inicial explorado
        actual_index = rand # Vecino a explorar
        while True:
            neighbour, actual_index = movement_operator(init_state, actual_solution, n_slots, neighbours_map, init_index, actual_index) # Movement_operator devuelve una lista
            
            # Debemos comprobar que no hemos llegado al final de la lista
            if actual_index != -1: # Si hemos encontrado un nuevo vecino
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones

                # Si el vecino encontrado es mejor que nuestra mejor solucion
                if best_neighbour_tkms > neighbour_tkms:
                    best_neighbour = neighbour.copy()
                    best_neighbour_tkms = neighbour_tkms
                    break # Estamos buscando el primer mejor vecino, por lo tanto, salimos del bucle
            else: # Si hemos explorado todos los vecinos dejamos de buscar
                break

        
        # Comprobamos si la solucion obtenida por el vecino es mejor que nuestra solucion actual
        if best_neighbour_tkms < actual_tkms: 
            # Guardamos al vecino como la nueva solucion actual y seguimos buscando
            actual_solution = best_neighbour.copy()
            actual_tkms = best_neighbour_tkms
            if display == True:
                print('New LS best solution: {} - {} kms - {} slots'.format(actual_solution, actual_tkms, sum(actual_solution)))
        else:
            best_neighbour_finded = False
            
        i += 1 # Nueva iteracion
        
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
       
    return [actual_solution, actual_tkms, ev]        
                

## Búsqueda de Entorno Variable (VNS: Variable Neighborhood Search)

La búsqueda de entorno variable es una metaherística para resolver problemas de optimización cuya idea básica es el cambio sistemático de entorno dentro de una búsqueda local.

* Sea E_k(k=1, .., k_max) un conjunto finito de estructuras de vecindario(entorno) preseleccionadas, y sea E_k(S) el conjunto de soluciones del entorno k-ésimo de S
* VNS aplica progradivamente una BL sobre una solución S' obtenida a partir de una mutación de la actual S, realizada de acuerdo el tipode entorno utilizado en cada iteración E_k(S)
    - Si la última BL afectuada resultó efectiva, es decir, si la solución obtenida tras ella, S'' mejorró la solución actual, S, se mantiene el entorno.
    - En caso contrario, se pasa al siguiente entorno (k<-k+1) pra provocar una pertuarbación mayor y modificar las soluciones que surgen de la BL desde S', de la zona del espacio de búsqueda en la que está situada la actual S y sus soluciones derivadas optimizadas con la BL

![title](img/vns.png)

In [148]:
def vns(seed, max_slots, n_slots, bl_max, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion usando el algoritmo de Busqueda de entorno variable (VNS). Para ello, una vez generada la solucion inicial. Se mutara la solucion y se realizara una busqueda local
    sobre la misma. Cuando se encuentre una nueva mejor solucion, esta sera la nueva solucion a mutar. El algoritmo terminar cuando se han explorado un numero de soluciones determinadas(bl_max)
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        bl_max : int 
            Numero maximo de nuevos vecinos a explorar. bl_max significa: busquedas locales maxima. Es asi porque cada vez que generamos un nuevo vecino realizamos una busqueda local en el mismo
        init_state : Numpy ndarray
            Array con el numero de slots minimo que debe tener cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    kms_history = np.array([])
    slots_history = np.array([])
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    if display == True:
        print('VNS init solution: {} - {} kms - {} slots'.format(init_solution, init_tkms, sum(init_solution)))
    
    # Inicializamos las variables
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_neighbour = None
    best_neighbour_tkms = None
    #best_tkms = None

    
    # Numero de vecindarios
    s = 0 # Tamanio de la sublista para crear vecinos. Cuando k = 1, s = 4. Cada vez que k aumente de valor, s valdra un multiplo de 4
    k = 1
    k_max = 4
    
    # Variable de control
    bl = 0 # Numero de busquedas locales realizadas
    
    # Comenzamos VNS
    while bl <= bl_max: # k tomara los valores 1,2,3,4
        s = k * 4 # Numero de sublista a crear en el operador de movimiento
        neighbour = movement_sub_list(init_state, actual_solution, n_slots, s)
        best_neighbour, best_neighbour_tkms, ev_local = local_search_vns(seed, max_slots, n_slots, init_state, neighbour, move_list, index_m, kms_m, False)
        
        bl += 1 # Aumentamos el numero de busquedas locales realizadas
        ev += ev_local # Aumentamos el numero de evaluaciones totales realizadas
        
        if best_neighbour_tkms < actual_tkms:
            actual_solution = best_neighbour
            actual_tkms = best_neighbour_tkms
            if display == True:
                print('VNS bl {} new solution: {} - {} kms - {} slots'.format(bl, actual_solution, actual_tkms, sum(actual_solution)))
            
            k = 1
        else:
            k += 1
            
        # Comprobamos que k no sea mayor que k_max
        if k > k_max:
            k = 1

        # Guardamos los resultados en el historial
        kms_history = np.append(kms_history, best_neighbour_tkms)
        slots_history = np.append(slots_history, sum(best_neighbour))
        
        #if display == True:
        #        print(f"VNS count: {bl} - k: {k}")
                
    return [actual_solution, actual_tkms, ev, kms_history, slots_history]

## Algoritmo genético básico

Un algoritmo genético consiste en que a partir de una población inicial. Se seleccionan a los mejores individuos de la población para que desdencientes que hereden sus caracteristicas y así obtengan mejores resultados. Por último, estos hijos reemplazaran a la población anterior.

Usaremos un modelo estacionario, este método nos permitira tener una presión selectiva alta. Por lo que la convergencia será más rápida. Por otro lado, el modelo generacional peca de ser semejante a una busqueda aleatoria

![title](img/genetic_basic.png)

In [149]:
def basic_genetic(seed, population_size, max_generations, max_time, init_state, move_list, index_m, kms_m, display):
    """
    Metodo que implementa un algoritmo genetico basico. Emplea un modelo estacionario
    
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        population_size : int
            Tamanio de la poblacion a crear
        max_generations : int 
            Numero de generaciones maximas que realizara el algoritmo
        max_time : int
            Cantidad de segundos maxima que puede tardar el algoritmo
        init_state : Numpy ndarray
            Array con el numero de slots minimo que debe tener cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    np.random.seed(seed) # Inicializamos la semilla
    ev = 0 # Numero de evaluaciones realizadas
    time_start = time.time() # Momento en el que empieza a ejecutarse el algoritmo
    
    kms_history = np.array([])
    slots_history = np.array([])
    
    best_individual = None # Mejor individuo de la poblacion
    best_individual_kms = None # Kilometros recorridos por el mejor individuo de la poblacion
    init_solution = None
    
    # CREAMOS LA POBLACION INICIAL
    min_slots = 210 
    max_slots = 260
    population = generate_initial_population(init_state, min_slots, max_slots, population_size)
    
    # PARAMETROS DE SELECCION POR TORNEO
    k = round(population_size * 0.3) # Establecemos el numero de candidatos. Sera un 30% del tamanio de la poblacion
    L = 4 # Numero de cromosomas a seleccionar a partir de de la seleccion previa (k). Escogeremos los 4 mejores

    # EVALUAMOS LA POBLACION
    population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
    best_individual, best_individual_kms = select_best_individual(population, population_kms)
    ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos 
    
    init_solution = best_individual.copy()
    
    if display == True:
        print('AG init solution: {} - {} kms - {} slots'.format(best_individual, best_individual_kms, sum(best_individual)))
    
    # REALIZAMOS EL BUCLE HASTA QUE SE CUMPLA LA CONDICION DE PARADA
    i = 0
    t = time.time() - time_start # Tiempo actual
    while i < max_generations and t < max_time:
        # SELECCIONAMOS A LOS MEJORES MEDIANTE TORNEO
        population_selected = selection_by_tournament(population, population_kms, k, L, "best")
        
        # RECOMBINAMOS
        new_individuals = cross_population(population_selected, init_state)
        
        # MUTAMOS
        mutated_individuals = mutate_population(new_individuals, init_state)
        
        # REEMPLAZAMOS A LOS PEORES MEDIANTE TORNEO
        population = replace_population(population, population_kms, k, mutated_individuals)
        
        # EVALUAMOS A LA POBLACION
        population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
        best_individual, best_individual_kms = select_best_individual(population, population_kms)
        ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos
        
        if display == True:
            print('AG generation {} best: {} - {} kms - {} slots'.format(i, best_individual, best_individual_kms, sum(best_individual)))
            
        # Guardamos al mejor en el historial de resultados
        kms_history = np.append(kms_history, best_individual_kms)
        slots_history = np.append(slots_history, sum(best_individual))    
            
        i += 1
        t = time.time() - time_start
        
    return [best_individual, best_individual_kms, ev, kms_history, slots_history]

### Metodos auxiliares genético básico

In [150]:
def generate_initial_population(init_state, min_slots, max_slots, n):
    """
    Genera una nueva poblacion de individuos
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        min_slots : int
            Numero de slots minimo que puede tener un individuo
        max_slots : int
            Numero de slots maximo que puede tener un individu
        n : int
            Tamanio de la poblacion a crear
            
    Return
        Devolvemos una lista de individuos
    """
    
    population = np.array([])
    chromosome = None
    for i in range(0,n):
        slots = np.random.randint(min_slots, max_slots)
        chromosome = generate_initial_solution(init_state, slots) # Creamos un nuevo individuo
        if len(population) != 0: # Si la poblacion ya se ha creado
            population = np.append(population, [chromosome], axis=0)
        else: # Si la poblacion aun no existe
            population = np.array([chromosome])
            
    return population

In [151]:
def evaluate_population(move_list, init_state, population, index_m, kms_s):
    """
    Evalua a todos los elementos de una poblacion
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        population : Numpy ndarray
            Contiene la poblacion a evaluar
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        Devuelve una lista de resultados, cada posicion de la lista se corresponde con su posicion equivalente en la poblacion
    """

    population_tkms = np.array([]) # Inicializamos la lista de resultados
    for chromosome in population:
        kms = evaluate(move_list, init_state, chromosome, index_m, kms_m) # Evaluamos la solucion actual
        population_tkms = np.append(population_tkms, kms) # Aniadimos el valor a la lista de resultados
        #print(f"chromosoma: {chromosome} - {kms} kms")
        
    return population_tkms

In [152]:
def selection_by_tournament(population, population_kms, k, L, type_selection):
    """
    Selecciona a los mejores elementos de la poblacion mediante, seleccion por torneo. Tras escoger aleatoriamente k individuos, se escogeran los L mejores
    
    Parametros
        population : Numpy ndarray
            Lista de inviduos de una poblacion (lista de soluciones)
        population_kms : Numpy ndarray
            Lista de resultados para cada uno de los individuos de la poblacion
        k : int
            Numero de individuos a pre-seleccionar de forma aleatoria
        L : int
            Numero maximo de individuos a seleccionar del grupo pre-selectivo. Estos indivudos son los que presentan mejores resultados 
        type_selection : string
            Cadena de texto que indica el tipo de seleccion. Usaremos "best" para seleccionar los mejores individuos y la palabra "worst" para seleccionar a los peores
    Return
        Devuelve una lista de individuos
    """
    best = "best"
    worst = "worst"
    if type_selection != best and type_selection != worst:
        raise Exception("Tipo de seleccion no valido")
    
    # Comprobamos que el grupo de pre-seleccion es igual o mayor que la seleccion final
    if k<L:
        k = L
        
    population_size = len(population)
    rng = np.random.default_rng() # Objeto para crear elementos aleatorios
    rand_list = rng.choice(population_size,size=k,replace=False) # De una lista de entre 0 y el tamanio de la popblacion (0-population_size), escogemos k elementos (size=k), sin repetir (replace=False)
    
    selected_population = population[rand_list] # Seleccionamos k elementos
    selected_population_kms = population_kms[rand_list] # Seleccionamos los km de los k elementos
    
    # Seleccionamos los L mejores individuos
    best_selected = np.array([])
    for i in range(0,L):
        if type_selection == best:
            selection = selected_population_kms == min(selected_population_kms) # Comprobamos que posicion del array tiene el menor resultado (mejor cromosoma) 
        else:
            selection = selected_population_kms == max(selected_population_kms) # Comprobamos que posicion del array tiene el mayor resultado (peor cromosoma) 

        chromosome = selected_population[selection][0] # Escogemos el mejor cromosoma. El [selection] devuelve al individuo dentro de una lista([[chromosome]]). Es por eso que ponemos el [0] para coger el unico elemento de la lista
        
        # Lo aniadimos a la lista
        if len(best_selected) != 0:
            best_selected = np.append(best_selected, [chromosome], axis=0)
        else:
            best_selected = np.array([chromosome])
            
        
        if type_selection == best:
            # Eliminamos el mejor cromosoma igualando a infinito su puntacion de km en la lista. Como el mejor es el valor mas pequenio, si lo igualamos a infinito no lo volveremos a seleccionar
            selected_population_kms[selection] = float('inf') 
        else:
            # Eliminamos el peor cromosoma igualando a 0 su puntacion de km en la lista. Como el peor es el valor mas grande, si lo igualamos a 0 no lo volveremos a seleccionar
            selected_population_kms[selection] = 0.0
 
    return best_selected
    

In [153]:
def cross(chromosome_a, chromosome_b, init_state):
    """
    Cruza dos cromosomas(soluciones) para obtener dos cromosomas nuevos
    
    Parametros
        chromosome_a : Numpy ndarray
            Vector solucion a
        chromosome_b : Numpy ndarray
            Vector solucion b
        init_state : Numpy ndarray
            Estado inicial de las estaciones
    Return
        Devuelve una lista de dos valores donde cada posicion de la misma es un vector solucion
    """
    
    '''
    Para crear 2 cromosomas nuevos seleccionaremos dos puntos aleatorios en funcion del tamanio del cromosoma. Y despues,
    se cogera una parte del cromosoma a y otra del b para formar uno nuevo. Despues debemos comprobar que la opcion sea valida
    con el estado inicial y que el numero de slots minimo sea 205
    '''
    n_genes = len(chromosome_a) # Numero de genes (estaciones)
    take = int(n_genes / 2) # Numero de genes a seleccionar a partir de un punto (spot)
    division_spots = np.random.randint(0,n_genes,2)
    
    # Creamos los nuevos hijos (2)
    child_list = np.array([])
    for spot in division_spots:
        new_chromosome = np.arange(n_genes) # Creamos un nuevo cromosoma con el tamanio de los padres
        # Seleccionamos la parte izquierda del cromosoma a con respecto al punto, y lo mismo pero con la parte derecha del cromosoma b
        p = n_genes - spot 
        if p < take: # Comprobamos si al coger la parte derecha nos pasamos del tamanio del array
            new_chromosome[spot:] = chromosome_a[spot:] # Seleccionamos desde el spot hasta el final del array
            number = take - p # Establecemos el ultimo numero a coger desde el inicio
            new_chromosome[:number] = chromosome_a[:number] # Unimos
            
            new_chromosome[(spot-take):spot] = chromosome_b[(spot-take):spot] # Le restamos al spot 8 slots para coger el rango
        else:
            new_chromosome[spot:(spot+take)] = chromosome_a[spot:(spot+take)] # Desde el spot cogemos 8 hacia delante
            
            new_chromosome[:spot] = chromosome_b[:spot] # Cogemos desde el spot hacia atras (hasta el inicio del array)
            number = take - spot # Le restamos al numero total de slots el spot. A partir de ese numero hasta el final
            new_chromosome[(n_genes-number):] = chromosome_b[(n_genes-number):] # Unimos
            
        
        # Comprobamos que es compatible con el estado inicial
        new_chromosome = check_chromosome(new_chromosome, init_state)
    
        # Aniadimos el hijo generado a la lista de hijos
        if len(child_list) == 0:
            child_list = np.array([new_chromosome])
        else:
            child_list = np.append(child_list,[new_chromosome],axis=0)
    
    return [child_list[0], child_list[1]]
    

In [154]:
def cross_population(population, init_state):
    """
    Crea hijos a partir de todos los individuos de una poblacion. El numero de hijos a crear sera igual al tamanio de la poblacion
    
    Parametros
        population : Numpy ndarray
            Lista de individuos de la poblacion (lista de soluciones)
        init_state : Numpy ndarray
            Estado inicial de las estaciones
    
    Return
        Devuelve una lista con todos los hijos generados
    """
    # Nos aseguramos que el numero de padres sea par
    population_size = len(population)
    if population_size&2 != 0:
        raise Exception("Debe haber un numero par de individuos")
        
    couples = population_size/2 # Numero parejas que vamos a tener
    child_list = np.array([])
    
    '''
    Para cruzar a los padres seleccionaremos parejas de forma aleatoria. Para ello crearemos un vector con el orden en el que crearemos las parejas. Por ejemplo:
    Si tenemos 8 individuos en la poblacion. Crearemos un vector de 8 elementos con el orden de los padres
        
        Vector: [4,2,1,3,7,0,5,6]
        
    Con este vector obtendremos los siguientes padres:
        
        Parejas: (4,2) (1,3) (7,0) (5,6) 
    '''
    rng = np.random.default_rng() # Objeto para crear elementos aleatorios
    rand_list = rng.choice(population_size,size=population_size,replace=False) # De una lista de entre 0 y el tamanio de la popblacion (0-population_size), escogemos k elementos (size=k), sin repetir (replace=False)
    
    i = 0
    while i < population_size:
        # Seleccionamos a dos padres de la poblacion
        father_a = rand_list[i]
        father_b = rand_list[i+1]
        
        # Los cruzamos
        child_a, child_b = cross(population[father_a], population[father_b], init_state) 
                  
        # Los aniadimos a la lista
        if len(child_list) != 0:
            child_list = np.append(child_list, [child_a, child_b], axis=0)
        else:
            child_list = np.array([child_a, child_b])
            
        # Aumentamos i para seleccionar a los nuevos padres en la siguiente iteracion
        i += 2
    
    return child_list
    

In [155]:
def mutate(chromosome, init_state):
    """
    Metodo que permite mutar un cromosoma. Del 5% al 20% de los genes son cambiados en promedio
    
    Parametros
        chromosome : Numpy ndarray
            Vector solucion, donde cada posicion del array es el valor que debe tener cada estacion
        init_state : Numpy ndarray
            Estado inicial de las estaciones
    
    Return
        Vector solucion
    """
    max_porcentage = 0.2
    min_porcentage = 0.05
    # probabilidad de 12.5%
    n_genes = len(chromosome) # Numero de genes (estaciones)
    mutated_chromosome = chromosome.copy()
    
    # Entre 5% al 20% de los genes son mutados en promedio. 
    # Calculamos cuanto es el maximo numero (cuanto es el 20%) y el minimo numero (cuanto es el 5%) de genes a cambiar en funcion del numero de genes encontrados en el cromosoma
    n_change = np.random.randint(round(min_porcentage*n_genes),round(max_porcentage*n_genes))
    
    # Seleccionamos de forma aleatoria que genes van a ser modificados
    gene_list = np.random.randint(0, n_genes, n_change)
    
    # Alteramos los valores de cada gen
    max_variation = 1 # Numero de slots quitar o aniadir de media
    for gene in gene_list:
        # Para calcular el numero de slots a mutar, multiplicaremos el valor obtenido de una distribucion normal por una constante
        array_slots = np.random.standard_normal(1) * max_variation
        slots = round(array_slots[0])
        
        # Comprobamos que la mutacion es compatible con el estado inicial, en caso contrario, no se modifica el gen
        if (chromosome[gene] + slots) >= init_state[gene]:
            mutated_chromosome[gene] += slots
            
    # Nos aseguramos que el chromosoma es compatible con el estado inicial
    adjust_chromosome = adjust_solution(mutated_chromosome, init_state)
    
    return adjust_chromosome

In [156]:
def mutate_population(population, init_state):
    """
    Metodo que permite mutar todos los individuos de una poblacion
    
    Parametros
        population : Numpy ndarray
            Lista de individuos de la poblacion (lista de soluciones)
        init_state : Numpy ndarray
            Estado inicial de las estaciones
    
    Return
        Devuelve una lista con todos los individuos ya mutados
    """
    
    population_size = len(population)
    mutated_population = np.array([])
    
    # Mutamos todos los individuos de la poblacion
    for i in range(0,population_size):
        mutated_chromosome = mutate(population[i], init_state)
        
        if len(mutated_population) != 0:
            mutated_population = np.append(mutated_population, [mutated_chromosome], axis=0)
        else:
            mutated_population = np.array([mutated_chromosome])
    
    return mutated_population
    

In [157]:
def replace_population(population, population_kms, k, new_individuals):
    """
    Reemplaza a los peores elementos de la poblacion por mejores individuos
    
    Parametros
        population : Numpy ndarray
            Lista de inviduos de una poblacion (lista de soluciones)
        population_kms : Numpy ndarray
            Lista de resultados para cada uno de los individuos de la poblacion
        k : int
            Numero de individuos a pre-seleccionar de forma aleatoria
        new_individuals : Numpy ndarray
            Lista de individuos nuevos que reemplazaran a los peores
            
    Return
        Devuelve la poblacion actualizada
    """
    L = len(new_individuals) # Seleccionaremos tantos elementos a reemplazar como nuevos individuos tengamos
    
    # Seleccionamos a los peores elementos de la poblacion mediante torneo
    worst_population = selection_by_tournament(population, population_kms, k, L, "worst")
    
    population_size = len(population)
    new_population = population.copy()
    
    # Buscaremos la posicion en la que se encuentran los peores elementos de la poblacion y los reemplazaremos con los nuevos
    for i in range(0,L):
        for j in range(0,population_size):
            check = np.array_equal(population[j], worst_population[i]) # Comprueba si dos arrays son iguales
            
            if check == True:
                new_population[j] = new_individuals[i].copy() # Modificamos uno de los peores individuos por uno nuevo
                break
  
    return new_population    

In [158]:
def select_best_individual(population, population_kms):
    """
    Devuelve el mejor elemento de la poblacion
    
    Parametros
        population : Numpy ndarray
            Lista de inviduos de una poblacion (lista de soluciones)
        population_kms : Numpy ndarray
            Resultados de cada uno de los individuos de la poblacion
    Return
        Devuelve una lista de dos posiciones. En la primera se guarda el individuo y en la segunda el numero de kilometros del mismo
    """
    
    position = np.where(population_kms == min(population_kms)) # Buscamos la posicion del mejor individuo de la poblacion
    
    # Seleccionamos al mejor individuo
    best_individual = population[position][0]
    best_individual_kms = population_kms[position][0]
    
    return [best_individual, best_individual_kms]

## Algoritmo genético CHC

Aunque el algoritmo CHC fue concebido para coromosomas con codificación binaria, existen versiones para su uso con cromosomas con codificaición en vector. El cálculo de la distancia de Hamming se realizará teniendo en cuenta cuantos genes difieren entre sí(cuantas posiciones difieren). Sólo aquellas cadenas con una distancia (mayor a umbral) serán combinados. El umbral se inicializará a L/4 siendo **L la longitud de la cadena o cromosoma**. Cuando ningún descendiente es insertado en la nueva población el umbral se reduce a 1. 

En la fase de recombinación no se aplica ningún poreceso de mutación. En su lugar, cuando la población coverge o el preoceso de búsqeuda deja de progresar adecuadamente (el umbal de cruce llega a 0 y no se generan nuevos descenedientes), la población se reiniciará. El cromosoma que representa la mejor solución hasta ese momento se utilizará como patrón para generar la nueva población (copiándose), y el resto se inicializará de forma aleatoria.

La población debe ser menor o igual a la elegida en el genético básico. En el arranque, los volores de un cromosoma correspondedn al mejor individuo de la genereación anterior, y el resto serán aleatorios.

![title](img/chc.png)

In [159]:
def chc(seed, population_size, max_resets, radius, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion empleando un algoritmo genetico chc. El mismo escogera al mejor individuo generado en distintas poblaciones
    
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        population_size : int
            Tamanio de la poblacion a crear
        max_resets : int 
            Numero de veces que se creara una nueva poblacion para volver a buscar una nueva mejor solucion
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
        init_state : Numpy ndarray
            Array con el numero de slots minimo que debe tener cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    np.random.seed(seed) # Inicializamos la semilla
    ev = 0 # Numero de evaluaciones realizadas
    max_time = 300 # 1800s = 30min
    time_start = time.time() # Momento en el que empieza a ejecutarse el algoritmo
    n_elite = 5 # Tamanio de la elite
    elite = None # Mejor individuo de la poblacion
    elite_kms = None # Kilometros recorridos por el mejor individuo de la poblacion
    kms_history = np.array([])
    
    # PARAMETROS SELECCION DE CRUCE CON PREVENCION DE INCESTO
    n_stations = len(init_state)
    hamming_distance = n_stations/4 # El umbral se inicializa a l/4 (L es la longitud del cromosoma)
    
    # CREAMOS LA POBLACION INICIAL
    min_slots = 210 
    max_slots = 260
    population = generate_initial_population(init_state, min_slots, max_slots, population_size)
    
    # PARAMETROS DE SELECCION POR TORNEO
    k = round(population_size * 0.3) # Establecemos el numero de candidatos. Sera un 30% del tamanio de la poblacion
    L = 4 # Numero de cromosomas a seleccionar a partir de de la seleccion previa (k). Escogeremos los 4 mejores - ESTA L NO TIENE NADA QUE VER CON 'L' COMENTADA EN LA LINEA 12

    # EVALUAMOS LA POBLACION
    population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
    elite, elite_kms = pick_elite(population, population_kms, n_elite)
    ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos 
    
    if display == True:
        best_elite, best_elite_kms = select_best_individual(elite, elite_kms) # Seleccionamos al mejor individuo de la elite
        print('CHC init solution: {} - {} kms - {} slots'.format(best_elite, best_elite_kms, sum(best_elite)))
    
    # REALIZAMOS EL BUCLE HASTA QUE SE CUMPLA LA CONDICION DE PARADA - En este caso, un numero maximo de mutiarranques o se llegue al tiempo maximo (30 min)
    i = 0 # Indica el numero de generaciones llevadas a cabo 
    r = 0 # Indica el numero de resets realizados
    t = time.time() - time_start # Tiempo actual
    while r < max_resets and t < max_time:
        
        # RECOMBINAMOS USANDO A TODA LA POBLACION
        new_childs = incest_precaution_cross_population(population, init_state, hamming_distance, radius)
        
        if len(new_childs) != 0:  # Si se ha creado algun nuevo individuo
            # REEMPLAZAMOS A LOS PEORES MEDIANTE TORNEO
            population = replace_population(population, population_kms, k, new_childs)
            
            # EVALUAMOS A LA POBLACION
            population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
            best_individual, best_individual_kms = select_best_individual(population, population_kms)
            ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos

            if display == True:
                print('CHC gen({})_newChilds({}) best: {} - {} kms - {} slots'.format(i, len(new_childs), best_individual, best_individual_kms, sum(best_individual)))

            # COMPROBAMOS SI ALGUN INDIVIDUO DE LA NUEVA GENERACION ENTRA EN LA ELITE
            candidates, candidates_kms = pick_elite(population, population_kms, n_elite) # Seleccionamos a la elite de la generacion actual
            elite, elite_kms = update_elite(elite, elite_kms, candidates, candidates_kms) # Actualizamos la elite global
            
            # Actualizamos el historial de mejor km por generacion
            kms_history = np.append(kms_history, best_individual_kms)
            
        else: # En caso de que no se hayan podido crear nuevos hijos
            hamming_distance -= 1 # Restamos en 1 la distancia de hamming para que sean posibles, hijos mas parecidos a sus padres
            if display == True:
                print(f"Hamming distance = {hamming_distance}")
                
        # MUTIARRANQUE
        if hamming_distance == 0: # Si la distancia de hamming es igual a 0, significa que los hijos son identicos a sus padres. RESETEAMOS
            # Crearemos una nueva poblacion aleatoria, pero aniadiendo a la poblaciio la elite hasta el momento 
            size_aux = population_size - n_elite # Nuevos individuos aleatorios de la poblacion
            population = generate_initial_population(init_state, min_slots, max_slots, size_aux)
            
            population = np.append(population, elite, axis=0) # Le aniadimos a la lista de individuos creada, los individuos de elite
            np.random.shuffle(population) # Mezclamos los elementos de la nueva poblacion para que la elite se mezcle
            
            # Evaluamos a la nueva poblacion
            population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
            ev += population_size
            
            # Comprobamos si alguno puede entrar en la elite
            candidates, candidates_kms = pick_elite(population, population_kms, n_elite) # Seleccionamos a la elite de la generacion actual
            elite, elite_kms = update_elite(elite, elite_kms, candidates, candidates_kms) # Actualizamos la elite global
            best_elite, best_elite_kms = select_best_individual(elite, elite_kms)
            
            hamming_distance = n_stations/4 # El umbral se inicializa a l/4 (L es la longitud del cromosoma)
            r += 1 # Aumentamos el contador de resets
            

            print('CHC Reset {} best: {} - {} kms - {} slots'.format(r-1, best_elite, best_elite_kms, sum(best_elite)))
            
            if r < max_resets:
                print(f"POPULATION RESET {r} - HAMMING DISTANCE={hamming_distance}")
            
        i += 1
        t = time.time() - time_start
        
    best_elite, best_elite_kms = select_best_individual(elite, elite_kms) # Seleccionamos al mejor individuo de la elite
    return [best_elite, best_elite_kms, ev, r, kms_history, len(kms_history)]

### Metodos auxiliares algoritmo CHC

In [160]:
def pick_elite(population, population_kms, n_elite):
    """
    Selecciona a la elite de una población. Es decir, a la parte de la población que ha obtenido mejores resultados
    
    Parametros
        population : Numpy ndarray
            Lista de inviduos de una poblacion (lista de soluciones)
        population_kms : Numpy ndarray
            Resultados de cada uno de los individuos de la poblacion
        n_elite : int
            Numero de individuos a seleccionar como elite
            
    Return
        Devuelve una lista de dos posiciones. La primera contiene una lista con las soluciones, y la segunda una lista de los resultados de cada solucion
    """
    elite = np.array([])
    elite_kms = np.array([])
    population_kms_copy = population_kms.copy()
    
    for i in range(0,n_elite):
        position = np.where(population_kms_copy == min(population_kms_copy)) # Buscamos la posicion del mejor individuo de la poblacion
    
        # Seleccionamos al mejor individuo
        best_individual = population[position][0] # Ponemos [0] porque population[position] nos devuelve una lista con un solo individuo. Por eso tenemos que seleccionarlo
        best_individual_kms = population_kms[position][0]
        
        if len(elite) != 0:
            elite = np.append(elite, [best_individual], axis=0)
            elite_kms = np.append(elite_kms, best_individual_kms)
        else:
            elite = np.array([best_individual])
            elite_kms = np.array([best_individual_kms])
            
        population_kms_copy[population_kms_copy == best_individual_kms] = float('inf') 
        
    return [elite, elite_kms]

In [161]:
def update_elite(elite, elite_kms, candidates, candidates_kms):
    """
    Actualiza la lista de elite. Los candidatos entraran en la elite si son mejores que los mismos
    
    Parametros
        elite : Numpy ndarray
            Lista de inviduos considerados elite de una poblacion (lista de soluciones)
        elite_kms : Numpy ndarray
            Resultados de cada uno de los individuos de la elite
        candidates : Numpy ndarray
            Lista de inviduos considerados candidatos para entrar en la elite (lista de soluciones)
        candidates_kms : Numpy ndarray
            Resultados de cada uno de los candidatos
     
    Return
        Devuelve una lista de dos posiciones. La primera contiene una lista con las soluciones, y la segunda una lista de los resultados de cada solucion
    """
    new_elite = elite.copy()
    new_elite_kms = elite_kms.copy()
    n_candidates = len(candidates)
    for i in range(0, n_candidates):
        # Comprobamos si el peor individuo de la elite es mejor que el candidato
        elite_max = max(new_elite_kms)
        cand_kms = candidates_kms[i]
        
        if cand_kms < elite_max:
            position = np.where(new_elite_kms == elite_max) # Buscamos la posicion del mejor individuo de la poblacion
            
            new_elite[position] = candidates[i]
            new_elite_kms[position] = cand_kms
            
    return [new_elite, new_elite_kms] 

In [162]:
def incest_precaution_cross_population(population, init_state, hamming_distance, radius):  
    """
    Crea hijos a partir de todos los individuos de una poblacion. Para prevenir que dos individuos muy parecidos entre si tengan hijos, se usara la distancia de hamming como medida preventiva.
    El tamanio de la poblacion creada no tiene porque ser igual a la poblacion inicial
    
    Parametros
        population : Numpy ndarray
            Lista de individuos de la poblacion (lista de soluciones)
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        hamming_distance : int
            Indica el numero de genotipos a tener en cuenta para considerar a dos padres como iguales
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
            
    Return
        Devuelve una lista con los hijos generados
    """
    
    # Nos aseguramos que el numero de padres sea par
    population_size = len(population)
    if population_size%2 != 0:
        raise Exception("Debe haber un numero par de individuos")
        
    couples = population_size/2 # Numero parejas que vamos a tener
    child_list = np.array([])
    
    '''
    Para cruzar a los padres seleccionaremos parejas de forma aleatoria. Para ello crearemos un vector con el orden en el que crearemos las parejas. Por ejemplo:
    Si tenemos 8 individuos en la poblacion. Crearemos un vector de 8 elementos con el orden de los padres
        
        Vector: [4,2,1,3,7,0,5,6]
        
    Con este vector obtendremos los siguientes padres:
        
        Parejas: (4,2) (1,3) (7,0) (5,6) 
        
    DISTANCIA DE HAMMING
    Para determinar si dos padres son muy parecidos usaremos la distancia de hamming. En una primera instancia, la distancia sera 4. Esto significa, que debemos
    de comprobar si al menos 4 genotipos(posicion concreta en el array) de nuestros padres son diferentes entre si. Por ejemplo:
    
        Padre A: [1,1,2,1,3,4,3,4] - Padre B: [1,1,1,1,3,3,3,5]
        
    El padre A  y el padre B no podrian tener hijos porque solo difieren en 3 genotipos
    '''
    rng = np.random.default_rng() # Objeto para crear elementos aleatorios
    rand_list = rng.choice(population_size,size=population_size,replace=False) # De una lista de entre 0 y el tamanio de la popblacion (0-population_size), escogemos k elementos (size=k), sin repetir (replace=False)
    
    i = 0
    while i < population_size:
        # Seleccionamos a dos padres de la poblacion
        father_a = rand_list[i]
        father_b = rand_list[i+1]
        
        # Comprobamos que sean lo suficientemente diferentes
        if hamming_check(population[father_a], population[father_b], hamming_distance, radius):
            # Los cruzamos
            child_a, child_b = cross_blx_alpha(population[father_a], population[father_b], init_state, hamming_distance, radius) 

            # Los aniadimos a la lista
            if len(child_list) != 0:
                child_list = np.append(child_list, [child_a, child_b], axis=0)
            else:
                child_list = np.array([child_a, child_b])
            
        # Aumentamos i para seleccionar a los nuevos padres en la siguiente iteracion
        i += 2
    
    return child_list

In [163]:
def cross_blx_alpha(chromosome_a, chromosome_b, init_state, hamming_distance, radius):
    """
    Cruza dos cromosomas(soluciones) usando el metodo BLX-Alpha para obtener dos cromosomas nuevos. El mismo consiste en intercambiar la mitad de los genes entre ambos padres y mutarlos
    
    Parametros
        chromosome_a : Numpy ndarray
            Vector solucion a
        chromosome_b : Numpy ndarray
            Vector solucion b
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        hamming_distance : int
            Indica el numero de genotipos a tener en cuenta para considerar a dos padres como iguales
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
    Return
        Devuelve una lista de dos valores donde cada posicion de la misma es un vector solucion
    """
    import math
    
    # Seleccionamos (n_stations/2) genes. Intercambiamos sus valores, los del padre a al b y los del b al a. Despues mutamos el valor de slots a cada usando usando una distribucion gaussiana
    n_stations = len(chromosome_a)
    different = hamming_dist(chromosome_a, chromosome_b, radius)
    select = int(different/2) # Numero de genes a intercambiar |ceil redondea hacia arriba. Si el valor es 0.1-0.9 siempre redondea a 1
    rng = np.random.default_rng() # Objeto para crear elementos aleatorios
    rand_list = rng.choice(n_stations,size=n_stations,replace=False) # De una lista de entre 0 y el tamanio de la popblacion (0-population_size), escogemos k elementos (size=k), sin repetir (replace=False)
    
    # Creamos nuestros nuevos chromosomas
    ch_a = chromosome_a.copy()
    ch_b = chromosome_b.copy()
    
    # El objetivo es recorrer la lista de numeros aleatorios y buscar que posiciones cumplen la distancia de hamming. De ser asi se intercambian los valores
    for i in rand_list:
        # Si aun hay genes que intercambiar
        if select > 0:
            if hamming_check(chromosome_a, chromosome_b, hamming_distance, radius):
                # Intercambiamos los genes (estaciones)
                aux = ch_a[i]
                ch_a[i] = ch_b[i]
                ch_b[i] = aux

                # Ahora mutamos ambos valores con una distribucion gaussina
                ch_a[i] = int(np.random.normal(loc=ch_a[i],scale=1,size=1)) # Distribucion gaussiana(normal) - loc=x (media sobre el numero n) - scale=y (desviacion tipica) - size=z (cantidad de numeros a generar)
                ch_b[i] = int(np.random.normal(loc=ch_b[i],scale=1,size=1)) # Distribucion gaussiana(normal) - loc=x (media sobre el numero n) - scale=y (desviacion tipica) - size=z (cantidad de numeros a generar)
                
                select -= 1
        else:
            break
       
    # Comprobamos que ambos cromosomas sean compatibles con el estado inicial
    ch_a = check_chromosome(ch_a, init_state)
    ch_b = check_chromosome(ch_b, init_state)
    
    return [ch_a, ch_b]

In [164]:
def check_chromosome(chromosome, init_state):
    """
    Comprueba que todos los genes(estaciones) sean compatibles con el estado inicial. Primero comprueba que todos los genes sean compatibles con el estado inicial.
    En caso de no serlo, el gen tomara el mismo valor que el gen del estado inicial. Si a pesar de todo, el numero de slots que presenta el cromosoma es menor al minimo,
    se ajustara el valor de cada gen de forma proporcial al numero de slots minimos
    
    Parametros
        chromosome : Numpy ndarray
            Vector solucion
        init_state : Numpy ndarray
            Estado inicial de las estaciones
            
    Return 
        Devuelve el vector solucion ajustado al estado inicial en caso de que no fuese valido
    """
    # Comprobamos si alguna estacion tiene menos slots que el estado inicial
    check = np.where(chromosome < init_state)
    mutation_errors = len(check[0]) # Numero de genotipos que no son compatibles con el estado inicial
    new_chromosome = chromosome.copy()
    
    # Comprobamos si hay alguno que no sea compatible
    if mutation_errors > 0:
        # Corregimos los genes que no son compatibles
        for i in check:
            new_chromosome[i] = init_state[i]
                
    if new_chromosome.sum() < 205:
        proportion = 210 / new_chromosome.sum() # Nueva proporcion para el maximo de slots
        
        # station * proportion
        new_chromosome = np.around(new_chromosome * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
            
    return new_chromosome

In [165]:
def hamming_check(father_a, father_b, hamming_distance, radius):
    """
    Comprueba si dos padres tienen una distancia de hamming igual o mayor a la facilitada por parametro
    
    Parametros
        father_a : Numpy ndarray
            Individuo A (vector solucion)
        father_b : Numpy ndarray
            Individuo B (vector solucion)
        hamming_distance : int
            Numero entero que determina como de distintos son dos individuos. Concretamente, indica cuantos genotipos(estaciones en nuestro problema) son diferentes entre ambos padres
        radius : int
            Numero de slots de diferencia que deben tener como minimo dos genes, para ser considerados como iguales
    Return
        Devuelve True si los padres son los suficiente diferentes a la distancia de hamming solicitada, y False en caso contrario
    """
    
    check = False
    # Restamos cada par de genotipos y calculamos su valor absoluto. Despues, comprobamos si estan dentro del radio
    subtraction = np.absolute(father_a - father_b)
    # Comprobamos que genotipos son diferentes. Para serlo, su resta debe superar el tamanio del radio
    genotype_difference = sum(subtraction >= radius)
                              
    # Comprobamos si el numero de genotipos suficientes es igual o superior al minimo indicado
    if genotype_difference > hamming_distance:
        check = True
        
    return check

## Algoritmo Genético Multimodal

Considerando como partida el AG Básico, debe implementar un AG multimodal espacial
mediante el método de secuencial (5 nichos) o clearing, determinando un radio adecuado
basado en la distancia de hamming.

 * Donde d(i,j) es la distancia entre las soluciones.(distancia Hamming)
 * Tamaño de la población menor o igual al genético básico.
 * Probabilidad de cruce 0.8. (si no se cruza se copia como hijo)
 * Probabilidad de mutación por gen 0.01-0.05.
 
![title](./img/multimodal.png)
![title](./img/sequential_niche.png)

Las formulas empleadas son las siguientes:
    
   **G**(x,s_n-1) = { (d(x,s_n-1) / radius)^alpha   si (d(x,s_n-1) < radius | 1}
  
La función **G** nos dira cuanto debemos penalizar al resultado obtenido en función de como este de cerca nuestro cromosoma a las soluciones encontradas en diferentes nichos.

   * x: Nuevo individuo encontrado
   * s_n-1: Solucion nicho encontrada en ejecuciones previas
   * alpha(**beta**): Parametro que nos dirá cuanto tenemos que castigar a la solución. Si alpha=1, entonces la penalización será proporcional a los cerca que este. Sin embargo, si alpha=2, tendremos una penalizacion exponencial. De forma que cuanto más se este acercando el nicho, más será penalizado. **Para no confundir con la alpha mencionada en anteriores apartados lo llamaremos beta**
   * radius: radio que emplearemos en la distancia de hamming, nos dice como de lejos esta una solucion de otra
   * d(x, s_n-1): Distancia entre la solucion encontrada y la anterior
   
El problema de esta fórmula es que esta pensanda para maximizar, es decir, que castiga menos cuanto más se acerque a nichos previos. Por ello, usaremos la siguiente fórmula adaptada a nuestro problema:  

   **G**(x,s_n-1) = { 1 - (d(x,s_n-1) / radius)^beta   si (d(x,s_n-1) < radius | 0}
   
Finalmente, el valor de **G** multiplicara al fitness obtenido por nuestra solución

   fitness_final = fitness + fitness * **G**
   
Este proceso se repetira tantas veces como nichos tengamos:

   fitness_final = fitness + (fitness * **G**(x, nicho_1)) + (fitness * **G**(x, nicho_2)) + ... + (fitness * **G**(x, nicho_n))

In [166]:
def multimodal_genetic(seed, population_size, max_generations, n_niches, radius, radius_share, beta, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion empleando un algoritmo genetico multimodal.
    
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        population_size : int
            Tamanio de la poblacion a crear
        max_generations : int
            Numero maximo de generaciones realizara el algoritmo genetico basico
        n_niches : int 
            Numero de nichos a crear
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
        radius_share : int
            Indica cual es el radio de accion de un nicho. Es decir, si la distancia entre x y s es de 10 genes de diferencia(hay 10 genes que son considerados 
            diferentes, esto depende de radius) y la radius_share es igual a 8. No se aplicaria penalizacion porque esta fuera del rango de la solucion nicho. Sin
            embargo, si la distancia entre ambas fuera de 5 genes de diferencia. Al estar dentro del radio del nicho, se le aplicaria un castigo
        beta : float
            Valor para aumentar de forma exponencial el castigo aplicado. Si el valor es igual a 1, el castigo aplicado sera lineal en funcion de la cercania al nicho
            . Si aumentamos beta, el castigo se realizara de forma exponencial. Contra mas nos acerquemos a la solucion nicho, mas se castigara a nuestra solucion
        init_state : Numpy ndarray
            Array con el numero de slots minimo que debe tener cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    np.random.seed(seed) # Inicializamos la semilla
    niches = np.array([])
    niches_kms = np.array([])
    ev = 0
    max_time = 1800 # 1800s = 30min
    time_start = time.time()
    
    # REALIZAMOS EL BUCLE HASTA QUE SE CUMPLA LA CONDICION DE PARADA - En este caso, un numero maximo de nichos o se llegue al tiempo maximo (30 min)
    n = 0 # Indica el nicho actual
    t = time_start - time.time() # Tiempo actual
    while n < n_niches and t < max_time:
        # MOSTRAMOS POR PANTALLA EL NICHO EN EL QUE ESTAMOS
        print(f"NICHE {n}")
        
        # EJECUTAMOS UN ALGORITMO GENETICO BASICO PENALIZANDO LAS SOLUCIONES QUE ESTEN CERCA DE LOS NICHOS ENCONTRADOS
        solution, solution_kms, n_ev = multimodal_bg(population_size, max_generations, niches, radius, radius_share, beta, init_state, move_list, index_m, kms_m, display)
        ev += n_ev
        
        # MOSTRAMOS LA SOLUCION
        if display == True:
            print(f" Niche {n} best solution: {solution} - {solution_kms} kms - {sum(solution)} slots")
        
        # ANIADIMOS LA SOLUCION A LA LISTA DE NICHOS
        if len(niches) != 0:
            niches = np.append(niches, [solution], axis=0)
            niches_kms = np.append(niches_kms, solution_kms)
        else:
            niches = np.array([solution])
            niches_kms = np.append(niches_kms, solution_kms)
            
        n += 1
        t = time_start - time.time()
            
    best_solution, best_solution_kms = select_best_individual(niches, niches_kms)
    return [best_solution, best_solution_kms, ev, niches, niches_kms]

### Metodos auxiliares algoritmo genético multimodal

In [167]:
def multimodal_bg(population_size, max_generations, niches, radius, radius_share, beta, init_state, move_list, index_m, kms_m, display):
    """
    Metodo que implementa un algoritmo genetico basico. Esta ajustado para ser usado dentro del algoritmo genetico multimodal
    
    Parametros
        population_size : int
            Tamanio de la poblacion a crear
        max_generations : int 
            Numero de generaciones maximas que realizara el algoritmo
        niches : Numpy ndarray
            Lista de nichos previamente encontrados
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
        radius_share : int
            Indica cual es el radio de accion de un nicho. Es decir, si la distancia entre x y s es de 10 genes de diferencia(hay 10 genes que son considerados 
            diferentes, esto depende de radius) y la radius_share es igual a 8. No se aplicaria penalizacion porque esta fuera del rango de la solucion nicho. Sin
            embargo, si la distancia entre ambas fuera de 5 genes de diferencia. Al estar dentro del radio del nicho, se le aplicaria un castigo
        beta : float
            Valor para aumentar de forma exponencial el castigo aplicado. Si el valor es igual a 1, el castigo aplicado sera lineal en funcion de la cercania al nicho
            . Si aumentamos beta, el castigo se realizara de forma exponencial. Contra mas nos acerquemos a la solucion nicho, mas se castigara a nuestra solucion
        init_state : Numpy ndarray
            Array con el numero de slots minimo que debe tener cada estacion
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    
    
    ev = 0 # Numero de evaluaciones realizadas
    max_time = 600 # 1800s = 30min
    time_start = time.time() # Momento en el que empieza a ejecutarse el algoritmo
    
    best_individual = None # Mejor individuo de la poblacion
    best_individual_kms = None # Kilometros recorridos por el mejor individuo de la poblacion
    init_solution = None
    
    # CREAMOS LA POBLACION INICIAL
    min_slots = 210 
    max_slots = 260
    population = generate_initial_population(init_state, min_slots, max_slots, population_size)
    
    # PARAMETROS DE SELECCION POR TORNEO
    k = round(population_size * 0.3) # Establecemos el numero de candidatos. Sera un 30% del tamanio de la poblacion
    L = 4 # Numero de cromosomas a seleccionar a partir de de la seleccion previa (k). Escogeremos los 4 mejores

    # EVALUAMOS LA POBLACION
    population_kms = evaluate_population(move_list, init_state, population, index_m, kms_m)
    best_individual, best_individual_kms = select_best_individual(population, population_kms)
    ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos 
    
    init_solution = best_individual.copy()
    
    if display == True:
        print('  AG init solution: {} - {} kms - {} slots'.format(best_individual, best_individual_kms, sum(best_individual)))
    
    # REALIZAMOS EL BUCLE HASTA QUE SE CUMPLA LA CONDICION DE PARADA
    i = 0
    t = time.time() - time_start # Tiempo actual
    while i < max_generations and t < max_time:
        # SELECCIONAMOS A LOS MEJORES MEDIANTE TORNEO
        population_selected = selection_by_tournament(population, population_kms, k, L, "best")
        
        # RECOMBINAMOS
        new_individuals = cross_population(population_selected, init_state)
        
        # MUTAMOS
        mutated_individuals = mutate_population(new_individuals, init_state)
        
        # REEMPLAZAMOS A LOS PEORES MEDIANTE TORNEO
        population = replace_population(population, population_kms, k, mutated_individuals)
        
        # EVALUAMOS A LA POBLACION Y LA CASTIGAMOS SI ESTAN CERCA DE LOS NICHOS
        population_kms = evaluate_population_and_niches_punish(move_list, init_state, population, index_m, kms_m, niches, radius, radius_share, beta)
        best_individual, best_individual_kms = select_best_individual(population, population_kms)
        ev += population_size # Como en el metodo evaluate_population se han evaluado a todos los elementos de la poblacion. Sumamos tantas evaluaciones como individuos tengamos
        
        if display == True:
            print('    AG generation {} best: {} - {} kms - {} slots'.format(i, best_individual, best_individual_kms, sum(best_individual)))
            
        i += 1
        t = time.time() - time_start
        
    return [best_individual, best_individual_kms, ev]

In [168]:
def evaluate_population_and_niches_punish(move_list, init_state, population, index_m, kms_s, niches, radius, radius_share, beta):
    """
    Evalua a todos los elementos de una poblacion, castigando a los individuos que esten muy cerca de nichos encontrados previamente
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        population : Numpy ndarray
            Población a ser evaluada
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        niches : Numpy ndarray
            Lista de nichos previamente encontrados
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
        radius_share : int
            Indica cual es el radio de accion de un nicho. Es decir, si la distancia entre x y s es de 10 genes de diferencia(hay 10 genes que son considerados 
            diferentes, esto depende de radius) y la radius_share es igual a 8. No se aplicaria penalizacion porque esta fuera del rango de la solucion nicho. Sin
            embargo, si la distancia entre ambas fuera de 5 genes de diferencia. Al estar dentro del radio del nicho, se le aplicaria un castigo
        beta : float
            Valor para aumentar de forma exponencial el castigo aplicado. Si el valor es igual a 1, el castigo aplicado sera lineal en funcion de la cercania al nicho
            . Si aumentamos beta, el castigo se realizara de forma exponencial. Contra mas nos acerquemos a la solucion nicho, mas se castigara a nuestra solucion
    Returns
        Devuelve una lista de resultados, cada posicion de la lista se corresponde con su posicion equivalente en la poblacion
    """

    population_tkms = np.array([]) # Inicializamos la lista de resultados
    for chromosome in population:
        kms = evaluate(move_list, init_state, chromosome, index_m, kms_m) # Evaluamos la solucion actual
        kms_punish = kms # kms + castigos
        
        # APLICAMOS CASTIGO POR NICHOS
        for i in range(0,len(niches)):
            kms_punish = kms_punish + kms * g(population[i], niches[i], radius, radius_share, beta)
            
        population_tkms = np.append(population_tkms, kms) # Aniadimos el valor a la lista de resultados
        #print(f"chromosoma: {chromosome} - {kms} kms")
        
    return population_tkms

In [169]:
def g(x, niche_solution, radius, radius_share, beta):
    """
    La funcion g calcula un valor de castigo para la solucion x en funcion de como de cerca se encuentre de la solucion nicho. Si la solucion esta en el radio de la 
    solucion nicho, recibira un castigo en funcion de como de cerca este. En caso contrario no se aplicara ningun castigo
    
    Parametros
        x : Numpy ndarray
            Vector solucion
        s : Numpy ndarray
            Vector solucion
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
        radius_share : int
            Indica cual es el radio de accion de un nicho. Es decir, si la distancia entre x y s es de 10 genes de diferencia(hay 10 genes que son considerados 
            diferentes, esto depende de radius) y la radius_share es igual a 8. No se aplicaria penalizacion porque esta fuera del rango de la solucion nicho. Sin
            embargo, si la distancia entre ambas fuera de 5 genes de diferencia. Al estar dentro del radio del nicho, se le aplicaria un castigo
        beta : float
            Valor para aumentar de forma exponencial el castigo aplicado. Si el valor es igual a 1, el castigo aplicado sera lineal en funcion de la cercania al nicho
            . Si aumentamos beta, el castigo se realizara de forma exponencial. Contra mas nos acerquemos a la solucion nicho, mas se castigara a nuestra solucion
    Return
        Devuelve un valor entre 0-1. Este numero debera ser multiplicado por el fitness obtenido por la solucion x. De este forma, si x estaba fuera del rango de la 
        solucion nicho, g() devolvera 1. Conservando el valor total del resultado obtenido (fitness = fitness * 1). A medida que se acerque,
    """
    value = 0 # Solucion por defecto, siempre que d < radius. De esta forma, si la solucion esta lejos del nicho. No recibira ningun castigo
    d = hamming_dist(x, niche_solution, radius)

    if d < radius_share:
        value = 1 - (d / radius_share)**beta
        
    return value

In [170]:
def hamming_dist(x, s, radius):
    """
    Calcula la distancia a la que se encuentra la solucion x de la solucion s
    
    Parametros
        x : Numpy ndarray
            Vector solucion
        s : Numpy ndarray
            Vector solucion
        radius : int
            Indica cual es el numero de slots minimo para que dos genes sean considerados como iguales (gen=estacion)
            
    Return
        Devuelve la distancia a la que se encuentra
    """

    # Restamos cada par de genotipos y calculamos su valor absoluto. Despues, comprobamos si estan dentro del radio
    subtraction = np.absolute(x - s)
    # Comprobamos que genotipos son diferentes. Para serlo, su resta debe superar el tamanio del radio
    distance = sum(subtraction >= radius)
    
    return distance

# Test

## Busqueda local Test

In [171]:
def local_search_test(seed, move_list, init_state, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    n_slots = 1
    
    ls_solution = None # Inicializamos la mejor solucion a null
    ls_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    ls_solution, ls_tkms, ev = local_search(seed, max_slots, n_slots, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('    LS best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(ls_solution, ls_tkms, ev, total_time))
    return [ls_solution, ls_tkms, ev, total_time]

'''
import time
index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

display = True # Muestra el progreso de cada algoritmo en profundidad

solution, tkms, ev, total_time = local_search_test(1250, move_list, init_state, index_m, kms_m, display)
'''

"\nimport time\nindex_df = pd.read_csv('./bicicletas/cercanas_indices.csv')\nkms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')\ndeltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\n\ndisplay = True # Muestra el progreso de cada algoritmo en profundidad\n\nsolution, tkms, ev, total_time = local_search_test(1250, move_list, init_state, index_m, kms_m, display)\n"

## Busqueda de entorno variable (VNS) Test

In [172]:
def vns_test(seed, move_list, init_state, index_m, kms_m, display):
    #seed, max_slots, n_slots, bl_max, init_state, move_list, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    n_slots = 1
    bl_max = 60 # Numero maximo de busquedas locales a realizar
    
    solution = None # Inicializamos la mejor solucion a null
    tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    solution, tkms, ev, kms_history, slots_history = vns(seed, max_slots, n_slots, bl_max, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('    VNS best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(solution, tkms, ev, total_time))
    return [solution, tkms, ev, total_time, kms_history, slots_history]
'''
import time
index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

n_slots = 1 # Numero de slots a mover
display = True # Muestra el progreso de cada algoritmo en profundidad

solution, tkms, ev, total_time, kms_history, slots_history = vns_test(1250, n_slots, move_list, init_state, index_m, kms_m, display)
'''

"\nimport time\nindex_df = pd.read_csv('./bicicletas/cercanas_indices.csv')\nkms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')\ndeltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\n\nn_slots = 1 # Numero de slots a mover\ndisplay = True # Muestra el progreso de cada algoritmo en profundidad\n\nsolution, tkms, ev, total_time, kms_history, slots_history = vns_test(1250, n_slots, move_list, init_state, index_m, kms_m, display)\n"

## Busqueda genética básica Test

In [173]:
def basic_genetic_test(seed, move_list, init_state, index_m, kms_m, display):
    population_size = 24 # Tamanio maximo de la poblacion
    max_generations = 600 # Numero maximo de generaciones a crear
    max_time = 600 # 1800s = 30min
    solution = None # Inicializamos la mejor solucion a null
    tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    solution, tkms, ev, kms_history, slots_history = basic_genetic(seed, population_size, max_generations, max_time, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('    AG best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(solution, tkms, ev, str(datetime.timedelta(seconds=total_time))))
    return [solution, tkms, ev, total_time, kms_history, slots_history]
'''
import time
index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

display = True # Muestra el progreso de cada algoritmo en profundidad

solution, tkms, ev, total_time = basic_genetic_test(1250, move_list, init_state, index_m, kms_m, display)
'''

"\nimport time\nindex_df = pd.read_csv('./bicicletas/cercanas_indices.csv')\nkms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')\ndeltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\n\ndisplay = True # Muestra el progreso de cada algoritmo en profundidad\n\nsolution, tkms, ev, total_time = basic_genetic_test(1250, move_list, init_state, index_m, kms_m, display)\n"

## Búsqueda CHC Test

In [174]:
def chc_test(seed, move_list, init_state, index_m, kms_m, display):
    population_size = 24 # Tamanio maximo de la poblacion
    max_resets = 10 # Numero maximo de generaciones a crear
    radius = 12
    solution = None # Inicializamos la mejor solucion a null
    tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    solution, tkms, ev, n_resets, kms_history, n_generations = chc(seed, population_size, max_resets, radius, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('    CHC best solution: {} - {} km - Evaluaciones: {} - Slots {} - Tiempo empleado: {} s'.format(solution, tkms, ev, solution.sum(), str(datetime.timedelta(seconds=total_time))))
    return [solution, tkms, ev, total_time, n_resets, kms_history, n_generations]

'''
import time
import datetime
index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

display = False # Muestra el progreso de cada algoritmo en profundidad

solution, tkms, ev, total_time, n_resets, kms_history, n_generations = chc_test(1250, move_list, init_state, index_m, kms_m, display, 8)
'''

"\nimport time\nimport datetime\nindex_df = pd.read_csv('./bicicletas/cercanas_indices.csv')\nkms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')\ndeltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\n\ndisplay = False # Muestra el progreso de cada algoritmo en profundidad\n\nsolution, tkms, ev, total_time, n_resets, kms_history, n_generations = chc_test(1250, move_list, init_state, index_m, kms_m, display, 8)\n"

## Busqueda genética multimodal Test

In [175]:
def multimodal_genetic_test(seed, move_list, init_state, index_m, kms_m, display):
    population_size = 24 # Tamanio maximo de la poblacion
    max_generations = 500 # Numero maximo de generaciones a crear en cada nicho
    n_niches = 5 # Numero de nichos a crear
    radius_share = 4
    radius = 8
    beta = 1
    solution = None # Inicializamos la mejor solucion a null
    tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    # seed, population_size, max_generations, n_niches, radius_share, beta, init_state, move_list, index_m, kms_m, display
    start = time.time()
    solution, tkms, ev, niches, niches_kms = multimodal_genetic(seed, population_size, max_generations, n_niches, radius, radius_share, beta, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('MBG best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(solution, tkms, ev, str(datetime.timedelta(seconds=total_time))))
    return [solution, tkms, ev, total_time, niches, niches_kms]
'''
import time
index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

display = False # Muestra el progreso de cada algoritmo en profundidad

solution, tkms, ev, total_time = multimodal_genetic_test(1250, move_list, init_state, index_m, kms_m, display)
'''

"\nimport time\nindex_df = pd.read_csv('./bicicletas/cercanas_indices.csv')\nkms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')\ndeltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\n\ndisplay = False # Muestra el progreso de cada algoritmo en profundidad\n\nsolution, tkms, ev, total_time = multimodal_genetic_test(1250, move_list, init_state, index_m, kms_m, display)\n"

## Estudio del valor Alpha

In [176]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
#[1.2,1.6,2.0,2.4,2.8,3.2]
#exp[2.0,2.2,2.4,2.6,2.8,3.0]
alphas = [1.2,1.6,2.0,2.4,2.8,3.2]

name_file = "alpha_study_linear"
#df_columns = ["seed","alpha","kms_history","slots_history","best_kms","best_kms_slots"] # Columnas del DataFrame
df_columns = ["seed","alpha","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/alpha/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for a in alphas:
    for seed in seeds:
        print('SEMILLA({}) - {}'.format(i, seed))
        solution, tkms, ev, total_time, kms_history, slots_history = basic_genetic_test(seed, move_list, init_state, index_m, kms_m, display,a)
        km = evaluate_no_alpha(move_list,init_state,solution,index_m,kms_m)
        #row = np.array([seed, alpha, kms_history, slots_history, tkms, sum(solution)], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        row = np.array([seed, a, solution, km, solution.sum()], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        df.loc[len(df)] = row
        
        df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden
        
        i += 1
    i=0
    '''

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\n#[1.2,1.6,2.0,2.4,2.8,3.2]\n#exp[2.0,2.2,2.4,2.6,2.8,3.0]\nalphas = [1.2,1.6,2.0,2.4,2.8,3.2]\n\nname_file = "alpha_study_linear"\n#df_columns = ["seed","alpha","kms_history","slots_history","best_kms","best_kms_slots"] # Columnas del DataFrame\ndf_columns = ["seed","alpha","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame\ndf 

## Estudio genetico basico

In [177]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
population = [16,24,30]

name_file = "population_convergence_study"
df_columns = ["seed","population","kms_history","slots_history","best_kms","best_kms_slots"] # Columnas del DataFrame
#df_columns = ["seed","population","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/basic_genetic/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for a in population:
    for seed in seeds:
        print('SEMILLA({}) - {}'.format(i, seed))
        solution, tkms, ev, total_time, kms_history, slots_history = basic_genetic_test(seed, move_list, init_state, index_m, kms_m, display,a)
        row = np.array([seed, a, kms_history, slots_history, tkms, sum(solution)], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        df.loc[len(df)] = row
        
        df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden
        
        i += 1
    i=0
'''


'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\npopulation = [16,24,30]\n\nname_file = "population_convergence_study"\ndf_columns = ["seed","population","kms_history","slots_history","best_kms","best_kms_slots"] # Columnas del DataFrame\n#df_columns = ["seed","population","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame\ndf = pd.DataFrame(columns=df_columns) # Dataframe que

## Estudio multimodal

In [178]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
radius = [8,10,12]
radius_share = [4,6,8,10,12,14]

name_file = "multimodal_radius_study"
df_columns = ["seed","radius","radius_share","best_solution","best_solution_km","niches","niches_kms"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/multimodal/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for r in radius:
    for rs in radius_share:
        for seed in seeds:
            print('SEMILLA({}) - {}'.format(i, seed))
            solution, tkms, ev, total_time, niches, niches_kms = multimodal_genetic_test(seed, move_list, init_state, index_m, kms_m, display, r, rs)
            row = np.array([seed, r, rs, solution, tkms, niches, niches_kms], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
            df.loc[len(df)] = row

            df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden

            i += 1
        i=0
'''

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\nradius = [8,10,12]\nradius_share = [4,6,8,10,12,14]\n\nname_file = "multimodal_radius_study"\ndf_columns = ["seed","radius","radius_share","best_solution","best_solution_km","niches","niches_kms"] # Columnas del DataFrame\ndf = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion\ndf_path = \'./study/multimodal/\'+name_file+\'.csv\'\n#d

## Estudio CHC

In [179]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
radius = [4,6,8,10,12,14]

name_file = "chc_radius_study"
df_columns = ["seed","radius","n_resets","best_solution","best_solution_km","kms_history","n_generations"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/chc/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for r in radius:
    for seed in seeds:
        print('SEMILLA({}) - {}'.format(i, seed))
        solution, tkms, ev, total_time, n_resets, kms_history, n_generations = chc_test(seed, move_list, init_state, index_m, kms_m, display, r)
        row = np.array([seed, r, n_resets, solution, tkms, kms_history, n_generations], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        df.loc[len(df)] = row
        
        df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden
        
        i += 1
    i=0
'''

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\nradius = [4,6,8,10,12,14]\n\nname_file = "chc_radius_study"\ndf_columns = ["seed","radius","n_resets","best_solution","best_solution_km","kms_history","n_generations"] # Columnas del DataFrame\ndf = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion\ndf_path = \'./study/chc/\'+name_file+\'.csv\'\n#df = pd.read_csv(df_path)\n\ni=0\nfor

## Estudio sobre Radio

In [180]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
radius = [12]

name_file = "chc_radius_"+str(radius[0])+"_study"
df_columns = ["seed","index_kms_history","kms_history"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/chc/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for r in radius:
    for seed in seeds:
        print('SEMILLA({}) - {}'.format(i, seed))
        solution, tkms, ev, total_time, n_resets, kms_history, n_generations = chc_test(seed, move_list, init_state, index_m, kms_m, display, r)
        #row = np.array([seed, r, n_resets, solution, tkms, kms_history, n_generations], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        
        for j in range(0,len(kms_history)):
            row = np.array([seed, j, kms_history[j]], dtype=object)
            df.loc[len(df)] = row
        
        df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden
        
        i += 1
    i=0
'''

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\nradius = [12]\n\nname_file = "chc_radius_"+str(radius[0])+"_study"\ndf_columns = ["seed","index_kms_history","kms_history"] # Columnas del DataFrame\ndf = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion\ndf_path = \'./study/chc/\'+name_file+\'.csv\'\n#df = pd.read_csv(df_path)\n\ni=0\nfor r in radius:\n    for seed in seeds:\n     

## Estudio VNS

In [181]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds_test.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 
#seeds = np.array([1000])

display = False
bl_max = [20,40,60,80,100]

name_file = "max_bl_study"
df_columns = ["seed","n_bl","kms_history","slots_history","best_kms","best_kms_slots", "time"] # Columnas del DataFrame
#df_columns = ["seed","population","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './study/vns/'+name_file+'.csv'
#df = pd.read_csv(df_path)

i=0
for bl in bl_max:
    for seed in seeds:
        print('SEMILLA({}) - {}'.format(i, seed))
        solution, tkms, ev, total_time, kms_history, slots_history = vns_test(seed, move_list, init_state, index_m, kms_m, display,bl)
        row = np.array([seed, bl, kms_history, slots_history, tkms, sum(solution), total_time], dtype=object) # Con dtype(object) especificamos que vamos a meter datos de distintos tipos en el array
        df.loc[len(df)] = row
        
        df.to_csv(df_path, index=False) # Con index=False no se aniadira un fila extra con el orden
        
        i += 1
    i=0
'''

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport datetime\n\nindex_df = pd.read_csv(\'./bicicletas/cercanas_indices.csv\')\nkms_df = pd.read_csv(\'./bicicletas/cercanas_kms.csv\')\ndeltas_df = pd.read_csv(\'./bicicletas/deltas_5m_double.csv\')\nseeds_df = pd.read_csv(\'./seeds/seeds_test.csv\')\n\nindex_m = index_df.to_numpy() # index matrix\nkms_m = kms_df.to_numpy() # kms matrix\nmove_list = movements_to_list(deltas_df) # Lista de movimientos\ninit_state = deltas_df.iloc[0].to_numpy() # Estado inicial\nseeds = seeds_df.to_numpy()[:,1] # Semillas \n#seeds = np.array([1000])\n\ndisplay = False\nbl_max = [20,40,60,80,100]\n\nname_file = "max_bl_study"\ndf_columns = ["seed","n_bl","kms_history","slots_history","best_kms","best_kms_slots", "time"] # Columnas del DataFrame\n#df_columns = ["seed","population","best_solution","best_solution_kms","best_solution_slots"] # Columnas del DataFrame\ndf = pd.DataFrame(columns=df_columns) # Dataframe que almacena t

## Crear mapa

In [182]:
import folium
import numpy as np
import pandas as pd

def create_map_from_df_df(new_solution, init_solution):
    """
    Devuelve un objeto mapa a partir de una solucion y del estado inicial
    
    Parametros 
        new_solution : Object
            Resultado extraido de un pandas dataframe
        init_solution : Object
            Resultado inicial extraido de un pandas dataframe
    Return
        Devuelve un mapa a mostrar
    """
    str_solution = new_solution.iloc[0] 
    str_solution = str_solution.replace('[','')
    str_solution = str_solution.replace(']','')
    solution = np.fromstring(str_solution, dtype=int, sep=' ')
    
    init = init_solution.iloc[0] 
    init = init.replace('[','')
    init = init.replace(']','')
    init_s = np.fromstring(init, dtype=int, sep=' ')
    return create_map(solution, init_s)

def create_map_from_df(new_solution, init_solution):
    """
    Devuelve un objeto mapa a partir de una solucion y del estado inicial
    
    Parametros 
        new_solution : Object
            Resultado extraido de un pandas dataframe
        init_solution : Numpy ndarray
            Resultado inicial
    Return
        Devuelve un mapa a mostrar
    """
    # Praparamos la solucion
    str_solution = new_solution.iloc[0] 
    str_solution = str_solution.replace('[','')
    str_solution = str_solution.replace(']','')
    solution = np.fromstring(str_solution, dtype=int, sep=' ')
    
    return create_map(solution, init_solution)
    

def create_map(new_solution, init_solution):
    solution = new_solution
    
    # Leemos el archivo de posiciones
    positions = pd.read_csv('./bicicletas/positions.csv', sep=';')
    positions_m = positions.to_numpy()
    
    # Create map object
    l = [43.467935,-3.818877]
    m = folium.Map(location=l, zoom_start=13)
    
    # Range color
    n = 0
    color_red = '#e83845'
    color_green = '#008000'
    color_blue = '#428bca'
    color = None
    
    # Circle Marker
    n_station = len(init_solution)
    r = 1.2 # Radio
    difference = solution - init_solution # Miramos cuales han aumentado y cual reducido
    for i in range(0,n_station):
        # Seleccionamos color
        if difference[i] < n:
            color = color_red,
        elif difference[i] == n:
            color = color_green
        else:
            color = color_blue
        
        string = 'Estacion ' + str(i) + '\nActual '+str(solution[i])+'\n Inicial '+str(init_solution[i])
        l = positions_m[i,0:2].tolist()
        folium.CircleMarker(
            location = l,
            radius = int(r * abs(solution[i])),
            popup = string,
            color = color,
            fill = True,
            fill_color = color
        ).add_to(m)

        
    # Mostamos leyenda por texto
    print("LEYENDA")
    print(f"Color Rojo: Inferior al estado inicial (Diferencia<{n})")
    print(f"Color Verde: Similar al estado inicial (Diferencia={n})")
    print(f"Color azul: Superior al estado inicial (Diferencia>{n})")
    m.save('./map/map.html')
    return m

## Crear semillas

In [183]:
import numpy as np
import pandas as pd

def create_seeds(name_file, number):
    """
    Crea un fichero con semillas

    Parametros
        name_file : str
            Nombre que recibira el fichero. El fichero sera nombrado siguiendo el siguiente patron: 'name_file'.csv - Este sera guardado en la carpeta seeds
        number : int
            Numero de semillas a crear
    """
    max_value = 200000000

    name = name_file + '.csv'
    path = './seeds/' + name
    number_to_generate = number
    seed_list = None

    for i in range(0,number_to_generate):
        rand = np.random.randint(0, max_value)
        if i != 0:
            seed_list = np.append(seed_list, rand)
        else:
            seed_list = np.array([rand])

    #print('Semillas: {}'.format(seed_list))
    df = pd.DataFrame(seed_list)
    df.to_csv(path)

## Crear estudio

In [184]:
import pandas as pd
import numpy as np

def create_study(df):
    """
    Crea un el estudio de todos las ejecuciones encontradas en el DataFrame, separado por algoritmos
    
    Parametros 
        df : Pandas DataFrame
            DataFrame del cual estudiar la informacion
    """
    names_list = ['Local_search','VNS_Search','Basic_G','Multinode_G','CHC']
    global_columns = ['Algoritmo','Ev. Medias','Ev. Mejor','Desviacion tipica Ev','Mejor Kms','Media Kms', 'Desviacion tipica Kms']
    selec_columns = ['Semilla','Evaluaciones', 'Kilometros']
    
    # Estudio individual
    '''
    for i in range(0,len(names_list)):
        new_df = pd.DataFrame(columns=selec_columns)
        d = df[df['Algoritmo'] == names_list[i]]
        if len(d) > 0:
            new_df = algorithm_study(new_df, d, selec_columns)
            new_df.to_csv('./total/'+ names_list[i] +'.csv')
    '''
    '''
    # Busquead Aleatoria
    new_df = pd.DataFrame(columns=selec_columns)
    d = df[df['Algoritmo'] == names_list[1]]
    new_df = algorithm_study(new_df, d, selec_columns)
    new_df.to_csv('./estudio/Random_search.csv')
        
    # Busqueda Local
    new_df = pd.DataFrame(columns=selec_columns)
    d = df[df['Algoritmo'] == names_list[2]]
    new_df = algorithm_study(new_df, d, selec_columns)
    new_df.to_csv('./estudio/Local_search.csv')
    
    # Enfriamiento simulado
    new_df = pd.DataFrame(columns=selec_columns)
    d = df[df['Algoritmo'] == names_list[3]]
    new_df = algorithm_study(new_df, d, selec_columns)
    new_df.to_csv('./estudio/Simulated_annealing.csv')
    
    # Busqueda tabú
    new_df = pd.DataFrame(columns=selec_columns)
    d = df[df['Algoritmo'] == names_list[4]]
    new_df = algorithm_study(new_df, d, selec_columns)
    new_df.to_csv('./estudio/Tabu_search.csv')
    '''
    
    # GLOBAL
    new_df = pd.DataFrame(columns=global_columns)
    for i in range(0,len(names_list)):
        d = df[df['Algoritmo'] == names_list[i]]
        s = extract_result(d, names_list[i])
        new_df.loc[len(new_df)] = s
    new_df.set_index(global_columns[0])
    
    new_df.to_csv('./study/Global_results.csv', index=False)
    print('Estudio finalizado')

In [185]:
def algorithm_study(new_df, d, selec_columns):
    n_row = len(d)
    for i in range(0,n_row):
        #seed = d.at[i,'Semilla'] # Cogemos la fila i y la columna
        seed = d.iat[i,1] # Cogemos la fila i y la columna
        ev = d.iat[i,4] # Cogemos la fila y la columna
        km = d.iat[i,3] # Cogemos la fila y la columna
        s = np.array([seed,ev,km])
        new_df.loc[len(new_df)] = s
            
    new_df = new_df.set_index(selec_columns[0])
    return new_df

In [186]:
def extract_result(d, algorithm):
    ev_medias = np.mean(d['Evaluaciones'])
    ev_mejor = min(d['Evaluaciones'])
    ev_std = np.std(d['Evaluaciones'])
    km_mejor = min(d['Kilometros'])
    km_media = np.mean(d['Kilometros'])
    km_std = np.std(d['Kilometros'])
    
    return [algorithm, ev_medias, ev_mejor, ev_std, km_mejor, km_media, km_std]

# Main

### Seleccionar Semillas

In [187]:
name_file = 'seed'
#create_seeds(name_file, 5)

### Ejecutar

In [188]:
import numpy as np
import pandas as pd
import time
import datetime

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/'+name_file+'.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 

names_list = ['Local_search','VNS_Search','Basic_G','Multinode_G','CHC']

n_slots = 1 # Numero de slots a mover
display = False # Muestra el progreso de cada algoritmo en profundidad

# Variables auxiliares
init_solution = None
actual_solution = None
actual_tkms = None
ev = None
t = None

solution_list = np.array([]) # Lista de kilometros
tkms_list = np.array([])

df_columns = ['Semilla', 'Algoritmo', 'Kilometros', 'Evaluaciones', 'Solucion'] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './results/'+name_file+'.csv'


'''
ALGORITMOS
'''

i = 0
for seed in seeds:
    print('SEMILLA({}) - {}'.format(i, seed))
    
    # LOCAL SEACH
    print(names_list[0])
    actual_solution, actual_tkms, ev, t = local_search_test(seed, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[0], actual_tkms, ev, actual_solution)
    
    # VNS
    print(names_list[1])
    actual_solution, actual_tkms, ev, t, kms_history, slots_history = vns_test(seed, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[1], actual_tkms, ev, actual_solution)
    
    # BASIG GENETIC
    print(names_list[2])
    actual_solution, actual_tkms, ev, t, kms_history, slots_history = basic_genetic_test(seed, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[2], actual_tkms, ev, actual_solution)

    # MULTINODE
    print(names_list[3])
    actual_solution, actual_tkms, ev, t, niches, niches_kms = multimodal_genetic_test(seed, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[3], actual_tkms, ev, actual_solution)
    
    # CHC
    print(names_list[4])
    actual_solution, actual_tkms, ev, t, n_resets, kms_history, n_generations = chc_test(seed, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[4], actual_tkms, ev, actual_solution)
    
    print('')
    i += 1
    df.to_csv(df_path, index=False) # Guardamos el DataFrame
    
    
print('COMPLETADO')

SEMILLA(0) - 140078552
Local_search
    LS best solution: [ 6  7 13 12 14 13 10 15 11 15 20 21  8 13 16 26] - 607.748 km - Evaluaciones: 913 - Tiempo empleado: 4.074348449707031 s
VNS_Search
    VNS best solution: [ 8  7 13 12 17 13 11 12 10 14 16 23  8 13 19 24] - 586.029 km - Evaluaciones: 9082 - Tiempo empleado: 38.97340703010559 s
Basic_G
    AG best solution: [11 10 16 14 18 14 14  9 12 13 14 18  8 13 15 16] - 505.74 km - Evaluaciones: 14424 - Tiempo empleado: 0:01:07.086865 s
Multinode_G
NICHE 0
NICHE 1
NICHE 2
NICHE 3
NICHE 4
MBG best solution: [14 10 15 14 20 14 13 12  8 13 12 18 10 13 15 14] - 504.996 km - Evaluaciones: 60120 - Tiempo empleado: 0:04:47.245660 s
CHC
CHC Reset 0 best: [ 9  8 17  7 13 15 13 10 15 15 17 19 11 14 16 16] - 574.334 kms - 215 slots
POPULATION RESET 1 - HAMMING DISTANCE=4.0
CHC Reset 1 best: [ 9  8 17  7 13 15 13 10 15 15 17 19 11 14 16 16] - 574.334 kms - 215 slots
POPULATION RESET 2 - HAMMING DISTANCE=4.0
CHC Reset 2 best: [22  8 13 13 13 13 12 15  8

## Resultados

In [190]:
create_study(df)
results = pd.read_csv('./study/Global_results.csv')
results

,Algoritmo,Ev. Medias,Ev. Mejor,Desviacion tipica Ev,Mejor Kms,Media Kms,Desviacion tipica Kms
0,Local_search,678.8,391,195.417911,580.529,595.4316,10.396466
1,VNS_Search,8622.2,5240,2711.269474,577.342,582.1324,3.011575
2,Basic_G,14424.0,14424,0.000000,500.774,507.1040,4.737028
3,Multinode_G,60120.0,60120,0.000000,499.650,503.0090,2.079434
4,CHC,55819.2,46320,5055.459718,530.815,544.4964,13.641295


In [191]:
df = pd.read_csv('./results/'+name_file+'.csv')
best_km = df[df['Kilometros'] == min(df['Kilometros'])]
worst_km = df[df['Kilometros'] == max(df['Kilometros'])]
print('Mejor Resultado')
best_km

Mejor Resultado


,Semilla,Algoritmo,Kilometros,Evaluaciones,Solucion
18,63345587,Multinode_G,499.65,60120,[12 9 16 14 20 14 12 12 12 13 12 19 8 13 15 14]


In [192]:
best_solution = best_km['Solucion']
worst_solution = worst_km['Solucion']

print('Se comparan la mejor y la peor solución encontrada')
create_map_from_df_df(best_solution, worst_solution)

Se comparan la mejor y la peor solución encontrada
LEYENDA
Color Rojo: Inferior al estado inicial (Diferencia<0)
Color Verde: Similar al estado inicial (Diferencia=0)
Color azul: Superior al estado inicial (Diferencia>0)
